In [37]:
import pandas as pd
from pandas import Series, DataFrame
import csv
import matplotlib.pylab as pylab
import seaborn as sns
from datetime import *
import matplotlib.pylab as pylab
from pylab import *
mpl.rcParams['font.sans-serif'] = ['SimHei']
from sklearn.preprocessing import *
import numpy as np
from scipy.stats import mode
import matplotlib.dates
from sklearn.preprocessing import *
from sklearn import ensemble
from xgboost.sklearn import XGBClassifier
from sklearn.model_selection import GridSearchCV,cross_val_score
import matplotlib.pylab as plt
import xgboost as xgb
from sklearn import metrics
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.model_selection import train_test_split, KFold, GridSearchCV, StratifiedKFold
from sklearn.externals import joblib


In [2]:
print('~~~~~~~~~~~~~~开始导入数据~~~~~~~~~~~~~~~~~~~')

train_df_3 = pd.read_csv('/usr/competition/alimama/data/fusai/fusai_b_train_df_3_sample.csv')
train_df_2 = pd.read_csv('/usr/competition/alimama/data/fusai/fusai_b_train_df_2_sample.csv')
train_df_1 = pd.read_csv('/usr/competition/alimama/data/fusai/fusai_b_train_df_1_sample.csv')

print('~~~~~~~~~~~~~~导入数据完毕~~~~~~~~~~~~~~~~~~~')


~~~~~~~~~~~~~~开始导入数据~~~~~~~~~~~~~~~~~~~
~~~~~~~~~~~~~~导入数据完毕~~~~~~~~~~~~~~~~~~~


In [3]:
# 拆分多维度拼接的字段
def splitMultiFea(df):
    tempDf = df.drop_duplicates(subset=['item_id'])[['item_id','item_category_list','item_property_list']]
    tempDf['item_category_list_str'] = tempDf['item_category_list'].values
    tempDf['item_property_list_str'] = tempDf['item_property_list'].values
    tempDf['item_category_list'] = tempDf[tempDf.item_category_list.notnull()]['item_category_list'].map(lambda x: x.split(';'))
    tempDf['item_category0'] = tempDf[tempDf.item_category_list.notnull()]['item_category_list'].map(lambda x: x[0])
    tempDf['item_category1'] = tempDf[tempDf.item_category_list.notnull()]['item_category_list'].map(lambda x: x[1] if len(x)>1 else np.nan)
    tempDf['item_category2'] = tempDf[tempDf.item_category_list.notnull()]['item_category_list'].map(lambda x: x[2] if len(x)>2 else np.nan)
    tempDf['item_property_list'] = tempDf[tempDf.item_property_list.notnull()]['item_property_list'].map(lambda x: x.split(';'))
    df = df.drop(['item_category_list','item_property_list'], axis=1).merge(tempDf, how='left', on='item_id')
    df['item_prop_num'] = df['item_property_list'].dropna().map(lambda x: len(x))
    df['predict_category_property_str'] = df['predict_category_property'].values
    df['predict_category_property'] = df[df.predict_category_property.notnull()]['predict_category_property'].map(
        lambda x: {kv.split(':')[0]:((kv.split(':')[1].split(',') if kv.split(':')[1]!='-1' else []) if len(kv.split(':')) >= 2 else []) for kv in x.split(';')})
    return df

# 添加广告商品与查询词的相关性特征
def addContextFea(df):
    df['predict_category'] = df['predict_category_property'].dropna().map(lambda x: list(x.keys()))
    df['predict_cate_num'] = df['predict_category'].dropna().map(lambda x: len(x))
    idx = df[df.predict_category_property.notnull()].index
    df.loc[idx,'cate_intersect_num'] = list(map(lambda x: len(np.intersect1d(x[0],x[1])), df.loc[idx, ['item_category_list','predict_category']].values))
    df['predict_property'] = [set() for i in range(len(df))]
    idx = df[(df.item_category2.notnull())&(df.predict_category_property.notnull())].index
    df.loc[idx,'predict_property'] = list(map(lambda x: x[2]|set(x[1][x[0]]) if (x[0] in x[1].keys()) else x[2], df.loc[idx,['item_category2','predict_category_property','predict_property']].values))
    idx = df[(df.item_category1.notnull())&(df.predict_category_property.notnull())].index
    df.loc[idx,'predict_property'] = list(map(lambda x: x[2]|set(x[1][x[0]]) if (x[0] in x[1].keys()) else x[2], df.loc[idx,['item_category1','predict_category_property','predict_property']].values))
    df['predict_property'] = df['predict_property'].map(lambda x: np.nan if len(x)==0 else list(x))
    df['predict_prop_num'] = df[df.predict_property.notnull()]['predict_property'].map(lambda x: len(x))
    idx = df[(df.predict_property.notnull())&(df.item_property_list.notnull())].index
    df.loc[idx, 'prop_intersect_num'] = list(map(lambda x: len(np.intersect1d(x[0],x[1])), df.loc[idx, ['item_property_list','predict_property']].values))
    df.loc[idx,'prop_union_num'] = list(map(lambda x: len(np.union1d(x[0],x[1])), df.loc[idx, ['item_property_list','predict_property']].values))
    df['prop_jaccard'] = df['prop_intersect_num'] / df['prop_union_num']
    df['prop_predict_ratio'] = df['prop_intersect_num'] / df['predict_prop_num']
    df['prop_item_ratio'] = df['prop_intersect_num'] / df['item_prop_num']
    df.fillna({k:-1 for k in ['predict_prop_num','prop_intersect_num','prop_union_num','prop_jaccard','prop_predict_ratio','prop_item_ratio']}, inplace=True)
    return df

train_df_3 = splitMultiFea(train_df_3)
train_df_2 = splitMultiFea(train_df_2)
train_df_1 = splitMultiFea(train_df_1)

train_df_3 = addContextFea(train_df_3)
train_df_2 = addContextFea(train_df_2)
train_df_1 = addContextFea(train_df_1)


In [4]:
def addSet(setList):
    allSet = []
    for member in setList:
        allSet = allSet + member
    allSet = set(allSet)
    return allSet

#处理跟商品类目和属性相关的特征
def getCategoryFuture(df):
    df['predict_category_list'] = df['predict_category_property'].map(lambda x: [] if x == np.nan else list(kv.split(':')[0] for kv in str(x).split(';')))
    df['predict_category_set'] = df['predict_category_list'].map(lambda x: set(x))
    df['real_item_category_list'] = df['item_category_list'].map(lambda x: set(x))

    df['predict_property_list'] = df['predict_category_property'].map(lambda x: [] if x == np.nan else ((kv.split(':')[1].split(',') if kv.split(':')[1]!='-1' else []) if len(kv.split(':')) >= 2 else [] for kv in str(x).split(';')))
    df['predict_property_list'] = df['predict_property_list'].map(lambda x: addSet(x))
    df['item_property_list'] = df['item_property_list'].map(lambda x: set(x))
    return df

def getMatchProportion(df):
    match_category_proportion = []
    match_property_proportion = []
    for x,y,m,n in df[['real_item_category_list', 'predict_category_set', 'item_property_list', 'predict_property_list']].values:
        match_category = x & y
        match_property = m & n
        if len(y) > 0:
            category_proportion = len(match_category) / len(y)
            match_category_proportion.append(category_proportion)
        else:
            match_category_proportion.append(0)
        if len(n) > 0:
            property_proportion = len(match_property) / len(n)
            match_property_proportion.append(property_proportion)
        else:
            match_property_proportion.append(0)
    df['match_category_proportion'] = match_category_proportion
    df['match_property_proportion'] = match_property_proportion
    return df

#构造跟预测数目相关的特征
def getPredictNumber(df):
    df['predict_category_number'] = df['predict_category_set'].map(lambda x: len(x))
    df['predict_property_number'] = df['predict_property_list'].map(lambda x: len(x))
    return df

#构造跟类目预测精确性相关的特征
def getPredictAccuracy(df):
    isFirstCategoryIn = []
    isLastCategoryIn = []
    for x,y in df[['real_item_category_list', 'predict_category_list']].values:
        if y[0] in x:
            isFirstCategoryIn.append(1)
        else:
            isFirstCategoryIn.append(0)
        if y[len(y)-1] in x:
            isLastCategoryIn.append(1)
        else:
            isLastCategoryIn.append(0)
    df['isFirstCategoryIn'] = isFirstCategoryIn
    df['isLastCategoryIn'] = isLastCategoryIn
    return df

#添加商品属性个数以及类目个数特征
def getCPNumber(df):
    df['category_number'] = df['item_category_list'].map(lambda x: len(x))
    df['property_number'] = df['item_property_list'].map(lambda x: len(x))
    return df

train_df_3 = getCategoryFuture(train_df_3)
train_df_2 = getCategoryFuture(train_df_2)
train_df_1 = getCategoryFuture(train_df_1)

train_df_3 = getMatchProportion(train_df_3)
train_df_2 = getMatchProportion(train_df_2)
train_df_1 = getMatchProportion(train_df_1)

train_df_3 = getPredictNumber(train_df_3)
train_df_2 = getPredictNumber(train_df_2)
train_df_1 = getPredictNumber(train_df_1)

train_df_3 = getPredictAccuracy(train_df_3)
train_df_2 = getPredictAccuracy(train_df_2)
train_df_1 = getPredictAccuracy(train_df_1)

train_df_3 = getCPNumber(train_df_3)
train_df_2 = getCPNumber(train_df_2)
train_df_1 = getCPNumber(train_df_1)


In [5]:
#导入测试集进行数据处理
test_df = pd.read_csv('/usr/competition/alimama/data/fusai/fusai_b_test_df_sample.csv')
test_df['date'] = test_df.context_timestamp.map(lambda x: datetime.datetime.fromtimestamp(x))
test_df['weekday'] = test_df['date'].map(lambda x: x.weekday())
test_df['day'] = test_df['date'].map(lambda x: x.day)
test_df['hour'] = test_df['date'].map(lambda x: x.hour)
test_df = splitMultiFea(test_df)
test_df = addContextFea(test_df)
test_df = getCategoryFuture(test_df)
test_df = getMatchProportion(test_df)
test_df = getPredictNumber(test_df)
test_df = getPredictAccuracy(test_df)
test_df = getCPNumber(test_df)


In [6]:
#构造跟商品实际的根类目和叶子类目特征
train_df_1['real_first_category'] = train_df_1['item_category_list'].map(lambda x: np.nan if len(x) < 1 else x[0])
train_df_1['real_last_category'] = train_df_1['item_category_list'].map(lambda x: np.nan if len(x) < 1 else x[len(x) -1])
train_df_2['real_first_category'] = train_df_2['item_category_list'].map(lambda x: np.nan if len(x) < 1 else x[0])
train_df_2['real_last_category'] = train_df_2['item_category_list'].map(lambda x: np.nan if len(x) < 1 else x[len(x) -1])
train_df_3['real_first_category'] = train_df_3['item_category_list'].map(lambda x: np.nan if len(x) < 1 else x[0])
train_df_3['real_last_category'] = train_df_3['item_category_list'].map(lambda x: np.nan if len(x) < 1 else x[len(x) -1])
test_df['real_first_category'] = test_df['item_category_list'].map(lambda x: np.nan if len(x) < 1 else x[0])
test_df['real_last_category'] = test_df['item_category_list'].map(lambda x: np.nan if len(x) < 1 else x[len(x) -1])


In [7]:
print(train_df_3.columns.values)
print(len(train_df_3.columns.values))


['instance_id' 'item_id' 'item_brand_id' 'item_city_id' 'item_price_level'
 'item_sales_level' 'item_collected_level' 'item_pv_level' 'user_id'
 'user_gender_id' 'user_age_level' 'user_occupation_id' 'user_star_level'
 'context_id' 'context_timestamp' 'context_page_id'
 'predict_category_property' 'shop_id' 'shop_review_num_level'
 'shop_review_positive_rate' 'shop_star_level' 'shop_score_service'
 'shop_score_delivery' 'shop_score_description' 'is_trade' 'date' 'weekday'
 'day' 'hour' 'item_category_list' 'item_property_list'
 'item_category_list_str' 'item_property_list_str' 'item_category0'
 'item_category1' 'item_category2' 'item_prop_num'
 'predict_category_property_str' 'predict_category' 'predict_cate_num'
 'cate_intersect_num' 'predict_property' 'predict_prop_num'
 'prop_intersect_num' 'prop_union_num' 'prop_jaccard' 'prop_predict_ratio'
 'prop_item_ratio' 'predict_category_list' 'predict_category_set'
 'real_item_category_list' 'predict_property_list'
 'match_category_proporti

In [10]:
#删除某些不必要的列
drop_fea = ['item_category_list', 'item_property_list', 
            'item_category_list_str', 'item_property_list_str', 'item_category0', 
            'item_category1', 'item_category2', 'item_prop_num',
            'predict_category_property_str', 'predict_category', 'predict_cate_num',
            'cate_intersect_num', 'predict_property', 'predict_prop_num',
            'prop_intersect_num', 'prop_union_num', 'predict_category_list', 'predict_category_set',
            'real_item_category_list', 'predict_property_list']
train_df_1.drop(drop_fea, axis=1, inplace=True)
train_df_2.drop(drop_fea, axis=1, inplace=True)
train_df_3.drop(drop_fea, axis=1, inplace=True)
test_df.drop(drop_fea, axis=1, inplace=True)


ValueError: labels ['item_category_list' 'item_property_list' 'item_category_list_str'
 'item_property_list_str' 'item_category0' 'item_category1'
 'item_category2' 'item_prop_num' 'predict_category_property_str'
 'predict_category' 'predict_cate_num' 'cate_intersect_num'
 'predict_property' 'predict_prop_num' 'prop_intersect_num'
 'prop_union_num' 'predict_category_list' 'predict_category_set'
 'real_item_category_list' 'predict_property_list'] not contained in axis

In [9]:
print(train_df_1.columns.values)
print(len(train_df_3.columns.values))


['instance_id' 'item_id' 'item_brand_id' 'item_city_id' 'item_price_level'
 'item_sales_level' 'item_collected_level' 'item_pv_level' 'user_id'
 'user_gender_id' 'user_age_level' 'user_occupation_id' 'user_star_level'
 'context_id' 'context_timestamp' 'context_page_id'
 'predict_category_property' 'shop_id' 'shop_review_num_level'
 'shop_review_positive_rate' 'shop_star_level' 'shop_score_service'
 'shop_score_delivery' 'shop_score_description' 'is_trade' 'date' 'weekday'
 'day' 'hour' 'prop_jaccard' 'prop_predict_ratio' 'prop_item_ratio'
 'match_category_proportion' 'match_property_proportion'
 'predict_category_number' 'predict_property_number' 'isFirstCategoryIn'
 'isLastCategoryIn' 'category_number' 'property_number'
 'real_first_category' 'real_last_category']
42


In [11]:
def getBayesSmoothParam(origion_rate):
    origion_rate_mean = origion_rate.mean()
    origion_rate_var = origion_rate.var()
    alpha = origion_rate_mean / origion_rate_var * (origion_rate_mean * (1 - origion_rate_mean) - origion_rate_var)
    beta = (1 - origion_rate_mean) / origion_rate_var * (origion_rate_mean * (1 - origion_rate_mean) - origion_rate_var)
    print('origion_rate_mean : ', origion_rate_mean)
    print('origion_rate_var : ', origion_rate_var)
    print('alpha : ', alpha)
    print('beta : ', beta)
    return alpha, beta

# 缩放字段至0-1
def scalerFea(df, cols):
    scaler = MinMaxScaler()
    df[cols] = scaler.fit_transform(df[[cols]].values)
    return df,scaler

def getHistoryInfoByCol(train_df_normal, train_df_1, train_df_2, train_df_3, test_df, colName):
    train_df_pivot_table_all = pd.pivot_table(train_df_normal[['instance_id', colName]], index=[colName], values=['instance_id'], aggfunc=len)
    train_df_pivot_table_all.reset_index(inplace=True)
    train_df_pivot_table_all.rename(columns={'instance_id' : 'all_' + colName + '_click_number'}, inplace=True)
#     print(train_df_pivot_table_all.head(10))

    train_df_pivot_table_buy = pd.pivot_table(train_df_normal[['instance_id', colName]][train_df_normal.is_trade == 1], index=[colName], values=['instance_id'], aggfunc=len)
    train_df_pivot_table_buy.reset_index(inplace=True)
    train_df_pivot_table_buy.rename(columns={'instance_id' : 'all_' + colName + '_buy_number'}, inplace=True)
#     print(train_df_pivot_table_buy.head(10))

    train_df_pivot_table = pd.merge(train_df_pivot_table_all, train_df_pivot_table_buy, on=[colName], how='left')
    train_df_pivot_table['all_' + colName + '_buy_number'] = train_df_pivot_table['all_' + colName + '_buy_number'].fillna(0)
    train_df_pivot_table['all_' + colName + '_buy_number'][train_df_pivot_table[colName] == -1] = train_df_pivot_table['all_' + colName + '_buy_number'][train_df_pivot_table[colName] == -1] / len(train_df_pivot_table)
    train_df_pivot_table['all_' + colName + '_click_number'][train_df_pivot_table[colName] == -1] = train_df_pivot_table['all_' + colName + '_click_number'][train_df_pivot_table[colName] == -1] / len(train_df_pivot_table)
#     print(train_df_pivot_table['all_' + colName + '_click_number'][train_df_pivot_table.item_brand_id == -1])
    train_df_pivot_table['history_' + colName + '_rate'] = train_df_pivot_table['all_' + colName + '_buy_number'] / train_df_pivot_table['all_' + colName + '_click_number']
    alpha, beta = getBayesSmoothParam(train_df_pivot_table['history_' + colName + '_rate'])
    train_df_pivot_table['history_' + colName + '_smooth_rate'] = (train_df_pivot_table['all_' + colName + '_buy_number'] + alpha) / (train_df_pivot_table['all_' + colName + '_click_number'] + alpha + beta)

    train_df_pivot_table, all_buy_number_scaler = scalerFea(train_df_pivot_table, 'all_' + colName + '_buy_number')
    train_df_pivot_table, all_click_number_scaler = scalerFea(train_df_pivot_table, 'all_' + colName + '_click_number')
#     print(train_df_pivot_table.head(10))
#     print(train_df_pivot_table.columns.values)

    train_df_1 = pd.merge(train_df_1, train_df_pivot_table, on=[colName], how='left')
    train_df_2 = pd.merge(train_df_2, train_df_pivot_table, on=[colName], how='left')
    train_df_3 = pd.merge(train_df_3, train_df_pivot_table, on=[colName], how='left')
    train_df_3['all_' + colName + '_click_number'] = train_df_3['all_' + colName + '_click_number'].fillna(0)
    train_df_3['all_' + colName + '_buy_number'] = train_df_3['all_' + colName + '_buy_number'].fillna(0)
    train_df_3['history_' + colName + '_smooth_rate'] = train_df_3['history_' + colName + '_smooth_rate'].fillna((alpha / (alpha + beta)))

    test_df = pd.merge(test_df, train_df_pivot_table, on=[colName], how='left')
    test_df['all_' + colName + '_click_number'] = test_df['all_' + colName + '_click_number'].fillna(0)
    test_df['all_' + colName + '_buy_number'] = test_df['all_' + colName + '_buy_number'].fillna(0)
    test_df['history_' + colName + '_smooth_rate'] = test_df['history_' + colName + '_smooth_rate'].fillna((alpha / (alpha + beta)))

    return train_df_1, train_df_2, train_df_3, test_df

train_df_normal = pd.concat([train_df_1, train_df_2])
train_df_1, train_df_2, train_df_3, test_df = getHistoryInfoByCol(train_df_normal, train_df_1, train_df_2, train_df_3, test_df, 'item_brand_id')
train_df_1, train_df_2, train_df_3, test_df = getHistoryInfoByCol(train_df_normal, train_df_1, train_df_2, train_df_3, test_df, 'shop_id')
train_df_1, train_df_2, train_df_3, test_df = getHistoryInfoByCol(train_df_normal, train_df_1, train_df_2, train_df_3, test_df, 'item_id')

print('~~~~~~~~~~~~~~历史数据统计完毕~~~~~~~~~~~~~~~~~~~')

print(train_df_1[['instance_id', 'all_item_brand_id_click_number']].head(10))


/usr/local/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/usr/local/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


origion_rate_mean :  0.013086259688778887
origion_rate_var :  0.003236829406100776
alpha :  0.03912815971526574
beta :  2.9508904281641324
origion_rate_mean :  0.013128726741275971
origion_rate_var :  0.0035945894362571255
alpha :  0.03419254764073707
beta :  2.5702144382429464


/usr/local/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:444: DataConversionWarning: Data with input dtype int64 was converted to float64 by MinMaxScaler.
  warnings.warn(msg, DataConversionWarning)


origion_rate_mean :  0.009856723155881956
origion_rate_var :  0.00357251595321706
alpha :  0.017070339669942695
beta :  1.7147769893032816
~~~~~~~~~~~~~~历史数据统计完毕~~~~~~~~~~~~~~~~~~~
                instance_id  all_item_brand_id_click_number
0       8940337983247977520                        0.012114
1       4858474345954405241                        0.004865
2       5053378222821165852                        0.000173
3       8890718968941765524                        0.043056
4        579096116755484782                        0.008292
5       1749254003519401220                        0.004611
6       5019544380774408770                        0.000022
7       2094542978124861709                        0.005235
8       4337024072645272631                        0.000123
9       3647726773891282224                        0.004232
10      8776086985778794477                        0.001956
11      5455246581601763171                        0.042974
12       198903881603399236            

In [12]:
train_df_1['date'] = pd.to_datetime(train_df_1['date'])
train_df_2['date'] = pd.to_datetime(train_df_2['date'])
train_df_3['date'] = pd.to_datetime(train_df_3['date'])

# 统计过去一个小时某用户点击某个相同商品的次数
def getOneHourSameItemCount(train_df_1, train_df_2, train_df_3, test_df):
    print(len(train_df_1))
    train_df_1_copy = train_df_1[['user_id', 'item_id', 'date', 'instance_id']]
    train_df_2_copy = train_df_2[['user_id', 'item_id', 'date', 'instance_id']]
    train_df_3_copy = train_df_3[['user_id', 'item_id', 'date', 'instance_id']]
    test_df_copy = test_df[['user_id', 'item_id', 'date', 'instance_id']]
    train_df_1_copy['user_id_str'] = train_df_1_copy['user_id'].map(lambda x: str(x))
    train_df_1_copy['item_id_str'] = train_df_1_copy['item_id'].map(lambda x: str(x))
    train_df_1_copy['user_item_id'] = train_df_1_copy['user_id_str'] + train_df_1_copy['item_id_str']
    train_df_2_copy['user_id_str'] = train_df_2_copy['user_id'].map(lambda x: str(x))
    train_df_2_copy['item_id_str'] = train_df_2_copy['item_id'].map(lambda x: str(x))
    train_df_2_copy['user_item_id'] = train_df_2_copy['user_id_str'] + train_df_2_copy['item_id_str']
    train_df_3_copy['user_id_str'] = train_df_3_copy['user_id'].map(lambda x: str(x))
    train_df_3_copy['item_id_str'] = train_df_3_copy['item_id'].map(lambda x: str(x))
    train_df_3_copy['user_item_id'] = train_df_3_copy['user_id_str'] + train_df_3_copy['item_id_str']
    test_df_copy['user_id_str'] = test_df_copy['user_id'].map(lambda x: str(x))
    test_df_copy['item_id_str'] = test_df_copy['item_id'].map(lambda x: str(x))
    test_df_copy['user_item_id'] = test_df_copy['user_id_str'] + test_df_copy['item_id_str']
    temp_df = pd.concat([train_df_1_copy, train_df_2_copy, train_df_3_copy, test_df_copy])
    tempDf = pd.pivot_table(temp_df, index=['user_item_id','date'], values=['instance_id'], aggfunc=len)
    tempDf.columns = ['show']
    tempDf.reset_index(inplace=True)
    tempDf['last_user_item'] = tempDf['user_item_id'].shift(1)
    tempDf['last_user_item'] = tempDf['last_user_item']==tempDf['user_item_id']
    tempDf['last_show_time'] = tempDf['date'].shift(1)
    tempDf.loc[~tempDf.last_user_item, 'last_show_time'] = np.nan
    tempDf['ui_last_show_timedelta'] = tempDf['date'] - tempDf['last_show_time']
    tempDf['ui_last_show_timedelta'] = tempDf['ui_last_show_timedelta'].dt.seconds
    tempDf['ui_last_show_timedelta'].fillna(999999, inplace=True)
    hourShowList = []
    hourShowTemp = {}
    for same, dt, show in tempDf[['last_user_item','date','show']].values:
        if same:
            [hourShowTemp.pop(k) for k in list(hourShowTemp) if k<dt-timedelta(hours=1)]
            hourShowList.append(np.sum(list(hourShowTemp.values())))
            hourShowTemp[dt] = show
        else:
            hourShowList.append(0)
            hourShowTemp = {dt:show}
    tempDf['lastOneHour_sameItem_count'] = hourShowList
    train_df_1_copy = train_df_1_copy.merge(tempDf[['user_item_id', 'date', 'lastOneHour_sameItem_count']], how='left', on=['user_item_id', 'date'])
    train_df_1['lastOneHour_sameItem_count'] = train_df_1_copy['lastOneHour_sameItem_count']
    train_df_2_copy = train_df_2_copy.merge(tempDf[['user_item_id', 'date', 'lastOneHour_sameItem_count']], how='left', on=['user_item_id', 'date'])
    train_df_2['lastOneHour_sameItem_count'] = train_df_2_copy['lastOneHour_sameItem_count']
    train_df_3_copy = train_df_3_copy.merge(tempDf[['user_item_id', 'date', 'lastOneHour_sameItem_count']], how='left', on=['user_item_id', 'date'])
    train_df_3['lastOneHour_sameItem_count'] = train_df_3_copy['lastOneHour_sameItem_count']
    test_df_copy = pd.merge(test_df_copy, tempDf[['user_item_id', 'date', 'lastOneHour_sameItem_count']], on = ['user_item_id', 'date'], how='left')
    test_df['lastOneHour_sameItem_count'] = test_df_copy['lastOneHour_sameItem_count']
    print(len(train_df_1))
    return train_df_1, train_df_2, train_df_3, test_df

train_df_1, train_df_2, train_df_3, test_df = getOneHourSameItemCount(train_df_1, train_df_2, train_df_3, test_df)


622020


/usr/local/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if sys.path[0] == '':
/usr/local/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  del sys.path[0]
/usr/local/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http

622020


In [13]:
# 统计过去一个小时某用户点击同种根类目商品的次数
def getOneHourSameFirstCategoryCount(train_df_1, train_df_2, train_df_3, test_df):
    print(len(train_df_1))
    train_df_1_copy = train_df_1[['user_id', 'real_first_category', 'date', 'instance_id']]
    train_df_2_copy = train_df_2[['user_id', 'real_first_category', 'date', 'instance_id']]
    train_df_3_copy = train_df_3[['user_id', 'real_first_category', 'date', 'instance_id']]
    test_df_copy = test_df[['user_id', 'real_first_category', 'date', 'instance_id']]
    train_df_1_copy['user_id_str'] = train_df_1_copy['user_id'].map(lambda x: str(x))
    train_df_1_copy['real_first_category_str'] = train_df_1_copy['real_first_category'].map(lambda x: str(x))
    train_df_1_copy['user_real_first_category'] = train_df_1_copy['user_id_str'] + train_df_1_copy['real_first_category_str']
    train_df_2_copy['user_id_str'] = train_df_2_copy['user_id'].map(lambda x: str(x))
    train_df_2_copy['real_first_category_str'] = train_df_2_copy['real_first_category'].map(lambda x: str(x))
    train_df_2_copy['user_real_first_category'] = train_df_2_copy['user_id_str'] + train_df_2_copy['real_first_category_str']
    train_df_3_copy['user_id_str'] = train_df_3_copy['user_id'].map(lambda x: str(x))
    train_df_3_copy['real_first_category_str'] = train_df_3_copy['real_first_category'].map(lambda x: str(x))
    train_df_3_copy['user_real_first_category'] = train_df_3_copy['user_id_str'] + train_df_3_copy['real_first_category_str']
    test_df_copy['user_id_str'] = test_df_copy['user_id'].map(lambda x: str(x))
    test_df_copy['real_first_category_str'] = test_df_copy['real_first_category'].map(lambda x: str(x))
    test_df_copy['user_real_first_category'] = test_df_copy['user_id_str'] + test_df_copy['real_first_category_str']
    temp_df = pd.concat([train_df_1_copy, train_df_2_copy, train_df_3_copy, test_df_copy])
    tempDf = pd.pivot_table(temp_df, index=['user_real_first_category','date'], values=['instance_id'], aggfunc=len)
    tempDf.columns = ['show']
    tempDf.reset_index(inplace=True)
    tempDf['last_user_real_first_category'] = tempDf['user_real_first_category'].shift(1)
    tempDf['last_user_real_first_category'] = tempDf['last_user_real_first_category']==tempDf['user_real_first_category']
    tempDf['last_show_time'] = tempDf['date'].shift(1)
    tempDf.loc[~tempDf.last_user_real_first_category, 'last_show_time'] = np.nan
    tempDf['ui_last_show_timedelta'] = tempDf['date'] - tempDf['last_show_time']
    tempDf['ui_last_show_timedelta'] = tempDf['ui_last_show_timedelta'].dt.seconds
    tempDf['ui_last_show_timedelta'].fillna(999999, inplace=True)
    hourShowList = []
    hourShowTemp = {}
    for same, dt, show in tempDf[['last_user_real_first_category','date','show']].values:
        if same:
            [hourShowTemp.pop(k) for k in list(hourShowTemp) if k<dt-timedelta(hours=1)]
            hourShowList.append(np.sum(list(hourShowTemp.values())))
            hourShowTemp[dt] = show
        else:
            hourShowList.append(0)
            hourShowTemp = {dt:show}
    tempDf['lastOneHour_sameFirstCategory_count'] = hourShowList
    train_df_1_copy = train_df_1_copy.merge(tempDf[['user_real_first_category', 'date', 'lastOneHour_sameFirstCategory_count']], how='left', on=['user_real_first_category', 'date'])
    train_df_1['lastOneHour_sameFirstCategory_count'] = train_df_1_copy['lastOneHour_sameFirstCategory_count']
    train_df_2_copy = train_df_2_copy.merge(tempDf[['user_real_first_category', 'date', 'lastOneHour_sameFirstCategory_count']], how='left', on=['user_real_first_category', 'date'])
    train_df_2['lastOneHour_sameFirstCategory_count'] = train_df_2_copy['lastOneHour_sameFirstCategory_count']
    train_df_3_copy = train_df_3_copy.merge(tempDf[['user_real_first_category', 'date', 'lastOneHour_sameFirstCategory_count']], how='left', on=['user_real_first_category', 'date'])
    train_df_3['lastOneHour_sameFirstCategory_count'] = train_df_3_copy['lastOneHour_sameFirstCategory_count']
    test_df_copy = pd.merge(test_df_copy, tempDf[['user_real_first_category', 'date', 'lastOneHour_sameFirstCategory_count']], on = ['user_real_first_category', 'date'], how='left')
    test_df['lastOneHour_sameFirstCategory_count'] = test_df_copy['lastOneHour_sameFirstCategory_count']
    print(len(train_df_1))
    return train_df_1, train_df_2, train_df_3, test_df

train_df_1, train_df_2, train_df_3, test_df = getOneHourSameFirstCategoryCount(train_df_1, train_df_2, train_df_3, test_df)


622020


/usr/local/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/usr/local/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
/usr/local/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.py

622020


In [14]:
# 统计过去一个小时某用户点击同种叶子类目商品的次数
def getOneHourSameLastCategoryCount(train_df_1, train_df_2, train_df_3, test_df):
    print(len(train_df_1))
    train_df_1_copy = train_df_1[['user_id', 'real_last_category', 'date', 'instance_id']]
    train_df_2_copy = train_df_2[['user_id', 'real_last_category', 'date', 'instance_id']]
    train_df_3_copy = train_df_3[['user_id', 'real_last_category', 'date', 'instance_id']]
    test_df_copy = test_df[['user_id', 'real_last_category', 'date', 'instance_id']]
    train_df_1_copy['user_id_str'] = train_df_1_copy['user_id'].map(lambda x: str(x))
    train_df_1_copy['real_last_category_str'] = train_df_1_copy['real_last_category'].map(lambda x: str(x))
    train_df_1_copy['user_real_last_category'] = train_df_1_copy['user_id_str'] + train_df_1_copy['real_last_category_str']
    train_df_2_copy['user_id_str'] = train_df_2_copy['user_id'].map(lambda x: str(x))
    train_df_2_copy['real_last_category_str'] = train_df_2_copy['real_last_category'].map(lambda x: str(x))
    train_df_2_copy['user_real_last_category'] = train_df_2_copy['user_id_str'] + train_df_2_copy['real_last_category_str']
    train_df_3_copy['user_id_str'] = train_df_3_copy['user_id'].map(lambda x: str(x))
    train_df_3_copy['real_last_category_str'] = train_df_3_copy['real_last_category'].map(lambda x: str(x))
    train_df_3_copy['user_real_last_category'] = train_df_3_copy['user_id_str'] + train_df_3_copy['real_last_category_str']
    test_df_copy['user_id_str'] = test_df_copy['user_id'].map(lambda x: str(x))
    test_df_copy['real_last_category_str'] = test_df_copy['real_last_category'].map(lambda x: str(x))
    test_df_copy['user_real_last_category'] = test_df_copy['user_id_str'] + test_df_copy['real_last_category_str']
    temp_df = pd.concat([train_df_1_copy, train_df_2_copy, train_df_3_copy, test_df_copy])
    tempDf = pd.pivot_table(temp_df, index=['user_real_last_category','date'], values=['instance_id'], aggfunc=len)
    tempDf.columns = ['show']
    tempDf.reset_index(inplace=True)
    tempDf['last_user_real_last_category'] = tempDf['user_real_last_category'].shift(1)
    tempDf['last_user_real_last_category'] = tempDf['last_user_real_last_category']==tempDf['user_real_last_category']
    tempDf['last_show_time'] = tempDf['date'].shift(1)
    tempDf.loc[~tempDf.last_user_real_last_category, 'last_show_time'] = np.nan
    tempDf['ui_last_show_timedelta'] = tempDf['date'] - tempDf['last_show_time']
    tempDf['ui_last_show_timedelta'] = tempDf['ui_last_show_timedelta'].dt.seconds
    tempDf['ui_last_show_timedelta'].fillna(999999, inplace=True)
    hourShowList = []
    hourShowTemp = {}
    for same, dt, show in tempDf[['last_user_real_last_category','date','show']].values:
        if same:
            [hourShowTemp.pop(k) for k in list(hourShowTemp) if k<dt-timedelta(hours=1)]
            hourShowList.append(np.sum(list(hourShowTemp.values())))
            hourShowTemp[dt] = show
        else:
            hourShowList.append(0)
            hourShowTemp = {dt:show}
    tempDf['lastOneHour_sameLastCategory_count'] = hourShowList
    train_df_1_copy = train_df_1_copy.merge(tempDf[['user_real_last_category', 'date', 'lastOneHour_sameLastCategory_count']], how='left', on=['user_real_last_category', 'date'])
    train_df_1['lastOneHour_sameLastCategory_count'] = train_df_1_copy['lastOneHour_sameLastCategory_count']
    train_df_2_copy = train_df_2_copy.merge(tempDf[['user_real_last_category', 'date', 'lastOneHour_sameLastCategory_count']], how='left', on=['user_real_last_category', 'date'])
    train_df_2['lastOneHour_sameLastCategory_count'] = train_df_2_copy['lastOneHour_sameLastCategory_count']
    train_df_3_copy = train_df_3_copy.merge(tempDf[['user_real_last_category', 'date', 'lastOneHour_sameLastCategory_count']], how='left', on=['user_real_last_category', 'date'])
    train_df_3['lastOneHour_sameLastCategory_count'] = train_df_3_copy['lastOneHour_sameLastCategory_count']
    test_df_copy = pd.merge(test_df_copy, tempDf[['user_real_last_category', 'date', 'lastOneHour_sameLastCategory_count']], on = ['user_real_last_category', 'date'], how='left')
    test_df['lastOneHour_sameLastCategory_count'] = test_df_copy['lastOneHour_sameLastCategory_count']
    print(len(train_df_1))
    return train_df_1, train_df_2, train_df_3, test_df

train_df_1, train_df_2, train_df_3, test_df = getOneHourSameLastCategoryCount(train_df_1, train_df_2, train_df_3, test_df)



622020


/usr/local/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/usr/local/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
/usr/local/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.py

622020


In [15]:
# 统计过去一个小时某用户点击同种品牌商品的次数
def getOneHourSameBrandCount(train_df_1, train_df_2, train_df_3, test_df):
    print(len(train_df_1))
    train_df_1_copy = train_df_1[['user_id', 'item_brand_id', 'date', 'instance_id']]
    train_df_2_copy = train_df_2[['user_id', 'item_brand_id', 'date', 'instance_id']]
    train_df_3_copy = train_df_3[['user_id', 'item_brand_id', 'date', 'instance_id']]
    test_df_copy = test_df[['user_id', 'item_brand_id', 'date', 'instance_id']]
    train_df_1_copy['user_id_str'] = train_df_1_copy['user_id'].map(lambda x: str(x))
    train_df_1_copy['brand_id_str'] = train_df_1_copy['item_brand_id'].map(lambda x: str(x))
    train_df_1_copy['user_brand_id'] = train_df_1_copy['user_id_str'] + train_df_1_copy['brand_id_str']
    train_df_2_copy['user_id_str'] = train_df_2_copy['user_id'].map(lambda x: str(x))
    train_df_2_copy['brand_id_str'] = train_df_2_copy['item_brand_id'].map(lambda x: str(x))
    train_df_2_copy['user_brand_id'] = train_df_2_copy['user_id_str'] + train_df_2_copy['brand_id_str']
    train_df_3_copy['user_id_str'] = train_df_3_copy['user_id'].map(lambda x: str(x))
    train_df_3_copy['brand_id_str'] = train_df_3_copy['item_brand_id'].map(lambda x: str(x))
    train_df_3_copy['user_brand_id'] = train_df_3_copy['user_id_str'] + train_df_3_copy['brand_id_str']
    test_df_copy['user_id_str'] = test_df_copy['user_id'].map(lambda x: str(x))
    test_df_copy['brand_id_str'] = test_df_copy['item_brand_id'].map(lambda x: str(x))
    test_df_copy['user_brand_id'] = test_df_copy['user_id_str'] + test_df_copy['brand_id_str']
    temp_df = pd.concat([train_df_1_copy, train_df_2_copy, train_df_3_copy, test_df_copy])
    tempDf = pd.pivot_table(temp_df, index=['user_brand_id','date'], values=['instance_id'], aggfunc=len)
    tempDf.columns = ['show']
    tempDf.reset_index(inplace=True)
    tempDf['last_user_brand_id'] = tempDf['user_brand_id'].shift(1)
    tempDf['last_user_brand_id'] = tempDf['last_user_brand_id']==tempDf['user_brand_id']
    tempDf['last_show_time'] = tempDf['date'].shift(1)
    tempDf.loc[~tempDf.last_user_brand_id, 'last_show_time'] = np.nan
    tempDf['ui_last_show_timedelta'] = tempDf['date'] - tempDf['last_show_time']
    tempDf['ui_last_show_timedelta'] = tempDf['ui_last_show_timedelta'].dt.seconds
    tempDf['ui_last_show_timedelta'].fillna(999999, inplace=True)
    hourShowList = []
    hourShowTemp = {}
    for same, dt, show in tempDf[['last_user_brand_id','date','show']].values:
        if same:
            [hourShowTemp.pop(k) for k in list(hourShowTemp) if k<dt-timedelta(hours=1)]
            hourShowList.append(np.sum(list(hourShowTemp.values())))
            hourShowTemp[dt] = show
        else:
            hourShowList.append(0)
            hourShowTemp = {dt:show}
    tempDf['lastOneHour_sameBrand_count'] = hourShowList
    train_df_1_copy = train_df_1_copy.merge(tempDf[['user_brand_id', 'date', 'lastOneHour_sameBrand_count']], how='left', on=['user_brand_id', 'date'])
    train_df_1['lastOneHour_sameBrand_count'] = train_df_1_copy['lastOneHour_sameBrand_count']
    train_df_2_copy = train_df_2_copy.merge(tempDf[['user_brand_id', 'date', 'lastOneHour_sameBrand_count']], how='left', on=['user_brand_id', 'date'])
    train_df_2['lastOneHour_sameBrand_count'] = train_df_2_copy['lastOneHour_sameBrand_count']
    train_df_3_copy = train_df_3_copy.merge(tempDf[['user_brand_id', 'date', 'lastOneHour_sameBrand_count']], how='left', on=['user_brand_id', 'date'])
    train_df_3['lastOneHour_sameBrand_count'] = train_df_3_copy['lastOneHour_sameBrand_count']
    test_df_copy = pd.merge(test_df_copy, tempDf[['user_brand_id', 'date', 'lastOneHour_sameBrand_count']], on = ['user_brand_id', 'date'], how='left')
    test_df['lastOneHour_sameBrand_count'] = test_df_copy['lastOneHour_sameBrand_count']
    print(len(train_df_1))
    return train_df_1, train_df_2, train_df_3, test_df

train_df_1, train_df_2, train_df_3, test_df = getOneHourSameBrandCount(train_df_1, train_df_2, train_df_3, test_df)


622020


/usr/local/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/usr/local/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
/usr/local/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.py

622020


In [16]:
# 统计过去一个小时某用户点击同种店铺商品的次数
def getOneHourSameShopCount(train_df_1, train_df_2, train_df_3, test_df):
    print(len(train_df_1))
    train_df_1_copy = train_df_1[['user_id', 'shop_id', 'date', 'instance_id']]
    train_df_2_copy = train_df_2[['user_id', 'shop_id', 'date', 'instance_id']]
    train_df_3_copy = train_df_3[['user_id', 'shop_id', 'date', 'instance_id']]
    test_df_copy = test_df[['user_id', 'shop_id', 'date', 'instance_id']]
    train_df_1_copy['user_id_str'] = train_df_1_copy['user_id'].map(lambda x: str(x))
    train_df_1_copy['item_shop_id_str'] = train_df_1_copy['shop_id'].map(lambda x: str(x))
    train_df_1_copy['user_item_shop_id'] = train_df_1_copy['user_id_str'] + train_df_1_copy['item_shop_id_str']
    train_df_2_copy['user_id_str'] = train_df_2_copy['user_id'].map(lambda x: str(x))
    train_df_2_copy['item_shop_id_str'] = train_df_2_copy['shop_id'].map(lambda x: str(x))
    train_df_2_copy['user_item_shop_id'] = train_df_2_copy['user_id_str'] + train_df_2_copy['item_shop_id_str']
    train_df_3_copy['user_id_str'] = train_df_3_copy['user_id'].map(lambda x: str(x))
    train_df_3_copy['item_shop_id_str'] = train_df_3_copy['shop_id'].map(lambda x: str(x))
    train_df_3_copy['user_item_shop_id'] = train_df_3_copy['user_id_str'] + train_df_3_copy['item_shop_id_str']
    test_df_copy['user_id_str'] = test_df_copy['user_id'].map(lambda x: str(x))
    test_df_copy['item_shop_id_str'] = test_df_copy['shop_id'].map(lambda x: str(x))
    test_df_copy['user_item_shop_id'] = test_df_copy['user_id_str'] + test_df_copy['item_shop_id_str']
    temp_df = pd.concat([train_df_1_copy, train_df_2_copy, train_df_3_copy, test_df_copy])
    tempDf = pd.pivot_table(temp_df, index=['user_item_shop_id','date'], values=['instance_id'], aggfunc=len)
    tempDf.columns = ['show']
    tempDf.reset_index(inplace=True)
    tempDf['last_user_item_shop_id'] = tempDf['user_item_shop_id'].shift(1)
    tempDf['last_user_item_shop_id'] = tempDf['last_user_item_shop_id']==tempDf['user_item_shop_id']
    tempDf['last_show_time'] = tempDf['date'].shift(1)
    tempDf.loc[~tempDf.last_user_item_shop_id, 'last_show_time'] = np.nan
    tempDf['ui_last_show_timedelta'] = tempDf['date'] - tempDf['last_show_time']
    tempDf['ui_last_show_timedelta'] = tempDf['ui_last_show_timedelta'].dt.seconds
    tempDf['ui_last_show_timedelta'].fillna(999999, inplace=True)
    hourShowList = []
    hourShowTemp = {}
    for same, dt, show in tempDf[['last_user_item_shop_id','date','show']].values:
        if same:
            [hourShowTemp.pop(k) for k in list(hourShowTemp) if k<dt-timedelta(hours=1)]
            hourShowList.append(np.sum(list(hourShowTemp.values())))
            hourShowTemp[dt] = show
        else:
            hourShowList.append(0)
            hourShowTemp = {dt:show}
    tempDf['lastOneHour_sameShop_count'] = hourShowList
    train_df_1_copy = train_df_1_copy.merge(tempDf[['user_item_shop_id', 'date', 'lastOneHour_sameShop_count']], how='left', on=['user_item_shop_id', 'date'])
    train_df_1['lastOneHour_sameShop_count'] = train_df_1_copy['lastOneHour_sameShop_count']
    train_df_2_copy = train_df_2_copy.merge(tempDf[['user_item_shop_id', 'date', 'lastOneHour_sameShop_count']], how='left', on=['user_item_shop_id', 'date'])
    train_df_2['lastOneHour_sameShop_count'] = train_df_2_copy['lastOneHour_sameShop_count']
    train_df_3_copy = train_df_3_copy.merge(tempDf[['user_item_shop_id', 'date', 'lastOneHour_sameShop_count']], how='left', on=['user_item_shop_id', 'date'])
    train_df_3['lastOneHour_sameShop_count'] = train_df_3_copy['lastOneHour_sameShop_count']
    test_df_copy = pd.merge(test_df_copy, tempDf[['user_item_shop_id', 'date', 'lastOneHour_sameShop_count']], on = ['user_item_shop_id', 'date'], how='left')
    test_df['lastOneHour_sameShop_count'] = test_df_copy['lastOneHour_sameShop_count']
    print(len(train_df_1))
    return train_df_1, train_df_2, train_df_3, test_df

train_df_1, train_df_2, train_df_3, test_df = getOneHourSameShopCount(train_df_1, train_df_2, train_df_3, test_df)


622020


/usr/local/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/usr/local/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
/usr/local/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.py

622020


In [17]:
# 获取是否是该用户在这1个小时内第一次点击这个商品的特征
def getIsOneHourFirstClickItem(train_df_1, train_df_2, train_df_3, test_df):
    print(len(train_df_1))
    train_df_1_copy = train_df_1[['user_id', 'item_id', 'date', 'instance_id']]
    train_df_2_copy = train_df_2[['user_id', 'item_id', 'date', 'instance_id']]
    train_df_3_copy = train_df_3[['user_id', 'item_id', 'date', 'instance_id']]
    test_df_copy = test_df[['user_id', 'item_id', 'date', 'instance_id']]
    train_df_1_copy['user_id_str'] = train_df_1_copy['user_id'].map(lambda x: str(x))
    train_df_1_copy['item_id_str'] = train_df_1_copy['item_id'].map(lambda x: str(x))
    train_df_1_copy['user_item_id'] = train_df_1_copy['user_id_str'] + train_df_1_copy['item_id_str']
    train_df_2_copy['user_id_str'] = train_df_2_copy['user_id'].map(lambda x: str(x))
    train_df_2_copy['item_id_str'] = train_df_2_copy['item_id'].map(lambda x: str(x))
    train_df_2_copy['user_item_id'] = train_df_2_copy['user_id_str'] + train_df_2_copy['item_id_str']
    train_df_3_copy['user_id_str'] = train_df_3_copy['user_id'].map(lambda x: str(x))
    train_df_3_copy['item_id_str'] = train_df_3_copy['item_id'].map(lambda x: str(x))
    train_df_3_copy['user_item_id'] = train_df_3_copy['user_id_str'] + train_df_3_copy['item_id_str']
    test_df_copy['user_id_str'] = test_df_copy['user_id'].map(lambda x: str(x))
    test_df_copy['item_id_str'] = test_df_copy['item_id'].map(lambda x: str(x))
    test_df_copy['user_item_id'] = test_df_copy['user_id_str'] + test_df_copy['item_id_str']
    temp_df = pd.concat([train_df_1_copy, train_df_2_copy, train_df_3_copy, test_df_copy])
    tempDf = pd.pivot_table(temp_df, index=['user_item_id','date'], values=['instance_id'], aggfunc=len)
    tempDf.columns = ['show']
    tempDf.reset_index(inplace=True)
    tempDf['last_user_item_id'] = tempDf['user_item_id'].shift(1)
    tempDf['last_user_item_id'] = tempDf['last_user_item_id']==tempDf['user_item_id']
    tempDf['last_show_time'] = tempDf['date'].shift(1)
    tempDf.loc[~tempDf.last_user_item_id, 'last_show_time'] = np.nan
    tempDf['ui_last_show_timedelta'] = tempDf['date'] - tempDf['last_show_time']
    tempDf['ui_last_show_timedelta'] = tempDf['ui_last_show_timedelta'].dt.seconds
    tempDf['ui_last_show_timedelta'].fillna(999999, inplace=True)
    hourShowList = []
    hourShowTemp = {}
    for same, dt, show in tempDf[['last_user_item_id','date','show']].values:
        if same:
            [hourShowTemp.pop(k) for k in list(hourShowTemp) if k<dt-timedelta(hours=1)]
            if len(hourShowTemp) > 0:
                hourShowList.append(0)
            else:
                hourShowList.append(1)
            hourShowTemp[dt] = show
        else:
            hourShowList.append(1)
            hourShowTemp = {dt:show}
    tempDf['isLastOneHour_firstClickItem'] = hourShowList
    train_df_1_copy = train_df_1_copy.merge(tempDf[['user_item_id', 'date', 'isLastOneHour_firstClickItem']], how='left', on=['user_item_id', 'date'])
    train_df_1['isLastOneHour_firstClickItem'] = train_df_1_copy['isLastOneHour_firstClickItem']
    train_df_2_copy = train_df_2_copy.merge(tempDf[['user_item_id', 'date', 'isLastOneHour_firstClickItem']], how='left', on=['user_item_id', 'date'])
    train_df_2['isLastOneHour_firstClickItem'] = train_df_2_copy['isLastOneHour_firstClickItem']
    train_df_3_copy = train_df_3_copy.merge(tempDf[['user_item_id', 'date', 'isLastOneHour_firstClickItem']], how='left', on=['user_item_id', 'date'])
    train_df_3['isLastOneHour_firstClickItem'] = train_df_3_copy['isLastOneHour_firstClickItem']
    test_df_copy = pd.merge(test_df_copy, tempDf[['user_item_id', 'date', 'isLastOneHour_firstClickItem']], on = ['user_item_id', 'date'], how='left')
    test_df['isLastOneHour_firstClickItem'] = test_df_copy['isLastOneHour_firstClickItem']
    print(len(train_df_1))
    return train_df_1, train_df_2, train_df_3, test_df

train_df_1, train_df_2, train_df_3, test_df = getIsOneHourFirstClickItem(train_df_1, train_df_2, train_df_3, test_df)



622020


/usr/local/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/usr/local/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
/usr/local/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.py

622020


In [18]:
# 统计某用户距离上次点击相同商品的时间
def getUserItemLastClickDeltaTime(train_df_1, train_df_2, train_df_3, test_df):

    print(len(train_df_1))
    train_df_1_copy = train_df_1[['user_id', 'item_id', 'date', 'instance_id']]
    train_df_2_copy = train_df_2[['user_id', 'item_id', 'date', 'instance_id']]
    train_df_3_copy = train_df_3[['user_id', 'item_id', 'date', 'instance_id']]
    test_df_copy = test_df[['user_id', 'item_id', 'date', 'instance_id']]
    train_df_1_copy['user_id_str'] = train_df_1_copy['user_id'].map(lambda x: str(x))
    train_df_1_copy['item_id_str'] = train_df_1_copy['item_id'].map(lambda x: str(x))
    train_df_1_copy['user_item_id'] = train_df_1_copy['user_id_str'] + train_df_1_copy['item_id_str']
    train_df_2_copy['user_id_str'] = train_df_2_copy['user_id'].map(lambda x: str(x))
    train_df_2_copy['item_id_str'] = train_df_2_copy['item_id'].map(lambda x: str(x))
    train_df_2_copy['user_item_id'] = train_df_2_copy['user_id_str'] + train_df_2_copy['item_id_str']
    train_df_3_copy['user_id_str'] = train_df_3_copy['user_id'].map(lambda x: str(x))
    train_df_3_copy['item_id_str'] = train_df_3_copy['item_id'].map(lambda x: str(x))
    train_df_3_copy['user_item_id'] = train_df_3_copy['user_id_str'] + train_df_3_copy['item_id_str']
    test_df_copy['user_id_str'] = test_df_copy['user_id'].map(lambda x: str(x))
    test_df_copy['item_id_str'] = test_df_copy['item_id'].map(lambda x: str(x))
    test_df_copy['user_item_id'] = test_df_copy['user_id_str'] + test_df_copy['item_id_str']
    temp_df = pd.concat([train_df_1_copy, train_df_2_copy, train_df_3_copy, test_df_copy])
    temp_df['user_id_str'] = temp_df['user_id'].map(lambda x: str(x))
    temp_df['item_id_str'] = temp_df['item_id'].map(lambda x: str(x))
    temp_df['user_item_id'] = temp_df['user_id_str'] + temp_df['item_id_str']
    tempDf = pd.pivot_table(temp_df, index=['user_item_id', 'date'], values=['instance_id'], aggfunc=len)
    tempDf.columns = ['show']
    tempDf.reset_index(inplace=True)
    tempDf['last_user_item_id'] = tempDf['user_item_id'].shift(1)
    tempDf['last_user_item_id'] = tempDf['last_user_item_id']==tempDf['user_item_id']
    tempDf['last_show_time'] = tempDf['date'].shift(1)
    tempDf.loc[~tempDf.last_user_item_id, 'last_show_time'] = np.nan
    historyShowList = []
    deltaTime = 99999999
    for same, dt, lastShowTime in tempDf[['last_user_item_id', 'date', 'last_show_time']].values:
        if same:
            deltaTime = (dt - lastShowTime) / np.timedelta64(1, 's')
            historyShowList.append(deltaTime)
            deltaTime = 99999999
        else:
            historyShowList.append(deltaTime)
    tempDf['userItem_lastClickDeltaTime'] = historyShowList
    train_df_1_copy = train_df_1_copy.merge(tempDf[['user_item_id', 'date', 'userItem_lastClickDeltaTime']], how='left', on=['user_item_id', 'date'])
    train_df_1['userItem_lastClickDeltaTime'] = train_df_1_copy['userItem_lastClickDeltaTime']
    train_df_2_copy = train_df_2_copy.merge(tempDf[['user_item_id', 'date', 'userItem_lastClickDeltaTime']], how='left', on=['user_item_id', 'date'])
    train_df_2['userItem_lastClickDeltaTime'] = train_df_2_copy['userItem_lastClickDeltaTime']
    train_df_3_copy = train_df_3_copy.merge(tempDf[['user_item_id', 'date', 'userItem_lastClickDeltaTime']], how='left', on=['user_item_id', 'date'])
    train_df_3['userItem_lastClickDeltaTime'] = train_df_3_copy['userItem_lastClickDeltaTime']
    test_df_copy = pd.merge(test_df_copy, tempDf[['user_item_id', 'date', 'userItem_lastClickDeltaTime']], on = ['user_item_id', 'date'], how='left')
    test_df['userItem_lastClickDeltaTime'] = test_df_copy['userItem_lastClickDeltaTime']
    print(len(train_df_1))

    return train_df_1, train_df_2, train_df_3, test_df

train_df_1, train_df_2, train_df_3, test_df = getUserItemLastClickDeltaTime(train_df_1, train_df_2, train_df_3, test_df)



622020


/usr/local/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
/usr/local/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # Remove the CWD from sys.path while we load stuff.
/usr/local/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

Se

622020


In [19]:
# 统计某用户距离上次点击相同品牌商品的时间
def getUserBrandLastClickDeltaTime(train_df_1, train_df_2, train_df_3, test_df):
    print(len(train_df_1))
    train_df_1_copy = train_df_1[['user_id', 'item_brand_id', 'date', 'instance_id']]
    train_df_2_copy = train_df_2[['user_id', 'item_brand_id', 'date', 'instance_id']]
    train_df_3_copy = train_df_3[['user_id', 'item_brand_id', 'date', 'instance_id']]
    test_df_copy = test_df[['user_id', 'item_brand_id', 'date', 'instance_id']]
    train_df_1_copy['user_id_str'] = train_df_1_copy['user_id'].map(lambda x: str(x))
    train_df_1_copy['brand_id_str'] = train_df_1_copy['item_brand_id'].map(lambda x: str(x))
    train_df_1_copy['user_brand_id'] = train_df_1_copy['user_id_str'] + train_df_1_copy['brand_id_str']
    train_df_2_copy['user_id_str'] = train_df_2_copy['user_id'].map(lambda x: str(x))
    train_df_2_copy['brand_id_str'] = train_df_2_copy['item_brand_id'].map(lambda x: str(x))
    train_df_2_copy['user_brand_id'] = train_df_2_copy['user_id_str'] + train_df_2_copy['brand_id_str']
    train_df_3_copy['user_id_str'] = train_df_3_copy['user_id'].map(lambda x: str(x))
    train_df_3_copy['brand_id_str'] = train_df_3_copy['item_brand_id'].map(lambda x: str(x))
    train_df_3_copy['user_brand_id'] = train_df_3_copy['user_id_str'] + train_df_3_copy['brand_id_str']
    test_df_copy['user_id_str'] = test_df_copy['user_id'].map(lambda x: str(x))
    test_df_copy['brand_id_str'] = test_df_copy['item_brand_id'].map(lambda x: str(x))
    test_df_copy['user_brand_id'] = test_df_copy['user_id_str'] + test_df_copy['brand_id_str']
    temp_df = pd.concat([train_df_1_copy, train_df_2_copy, train_df_3_copy, test_df_copy])
    tempDf = pd.pivot_table(temp_df, index=['user_brand_id', 'date'], values=['instance_id'], aggfunc=len)
    tempDf.columns = ['show']
    tempDf.reset_index(inplace=True)
    tempDf['last_user_item_brand_id'] = tempDf['user_brand_id'].shift(1)
    tempDf['last_user_item_brand_id'] = tempDf['last_user_item_brand_id']==tempDf['user_brand_id']
    tempDf['last_show_time'] = tempDf['date'].shift(1)
    tempDf.loc[~tempDf.last_user_item_brand_id, 'last_show_time'] = np.nan
    historyShowList = []
    deltaTime = 99999999
    for same, dt, lastShowTime in tempDf[['last_user_item_brand_id', 'date', 'last_show_time']].values:
        if same:
            deltaTime = (dt - lastShowTime) / np.timedelta64(1, 's')
            historyShowList.append(deltaTime)
            deltaTime = 99999999
        else:
            historyShowList.append(deltaTime)
    tempDf['userBrand_lastClickDeltaTime'] = historyShowList
    train_df_1_copy = train_df_1_copy.merge(tempDf[['user_brand_id', 'date', 'userBrand_lastClickDeltaTime']], how='left', on=['user_brand_id', 'date'])
    train_df_1['userBrand_lastClickDeltaTime'] = train_df_1_copy['userBrand_lastClickDeltaTime']
    train_df_2_copy = train_df_2_copy.merge(tempDf[['user_brand_id', 'date', 'userBrand_lastClickDeltaTime']], how='left', on=['user_brand_id', 'date'])
    train_df_2['userBrand_lastClickDeltaTime'] = train_df_2_copy['userBrand_lastClickDeltaTime']
    train_df_3_copy = train_df_3_copy.merge(tempDf[['user_brand_id', 'date', 'userBrand_lastClickDeltaTime']], how='left', on=['user_brand_id', 'date'])
    train_df_3['userBrand_lastClickDeltaTime'] = train_df_3_copy['userBrand_lastClickDeltaTime']
    test_df_copy = pd.merge(test_df_copy, tempDf[['user_brand_id', 'date', 'userBrand_lastClickDeltaTime']], on = ['user_brand_id', 'date'], how='left')
    test_df['userBrand_lastClickDeltaTime'] = test_df_copy['userBrand_lastClickDeltaTime']
    print(len(train_df_1))
    return train_df_1, train_df_2, train_df_3, test_df

train_df_1, train_df_2, train_df_3, test_df = getUserBrandLastClickDeltaTime(train_df_1, train_df_2, train_df_3, test_df)



622020


/usr/local/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/usr/local/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
/usr/local/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.py

622020


In [20]:
# 统计某用户距离上次点击相同店铺的时间
def getUserShopLastClickDeltaTime(train_df_1, train_df_2, train_df_3, test_df):
    print(len(train_df_1))
    train_df_1_copy = train_df_1[['user_id', 'shop_id', 'date', 'instance_id']]
    train_df_2_copy = train_df_2[['user_id', 'shop_id', 'date', 'instance_id']]
    train_df_3_copy = train_df_3[['user_id', 'shop_id', 'date', 'instance_id']]
    test_df_copy = test_df[['user_id', 'shop_id', 'date', 'instance_id']]
    train_df_1_copy['user_id_str'] = train_df_1_copy['user_id'].map(lambda x: str(x))
    train_df_1_copy['item_shop_id_str'] = train_df_1_copy['shop_id'].map(lambda x: str(x))
    train_df_1_copy['user_item_shop_id'] = train_df_1_copy['user_id_str'] + train_df_1_copy['item_shop_id_str']
    train_df_2_copy['user_id_str'] = train_df_2_copy['user_id'].map(lambda x: str(x))
    train_df_2_copy['item_shop_id_str'] = train_df_2_copy['shop_id'].map(lambda x: str(x))
    train_df_2_copy['user_item_shop_id'] = train_df_2_copy['user_id_str'] + train_df_2_copy['item_shop_id_str']
    train_df_3_copy['user_id_str'] = train_df_3_copy['user_id'].map(lambda x: str(x))
    train_df_3_copy['item_shop_id_str'] = train_df_3_copy['shop_id'].map(lambda x: str(x))
    train_df_3_copy['user_item_shop_id'] = train_df_3_copy['user_id_str'] + train_df_3_copy['item_shop_id_str']
    test_df_copy['user_id_str'] = test_df_copy['user_id'].map(lambda x: str(x))
    test_df_copy['item_shop_id_str'] = test_df_copy['shop_id'].map(lambda x: str(x))
    test_df_copy['user_item_shop_id'] = test_df_copy['user_id_str'] + test_df_copy['item_shop_id_str']
    temp_df = pd.concat([train_df_1_copy, train_df_2_copy, train_df_3_copy, test_df_copy])
    tempDf = pd.pivot_table(temp_df, index=['user_item_shop_id', 'date'], values=['instance_id'], aggfunc=len)
    tempDf.columns = ['show']
    tempDf.reset_index(inplace=True)
    tempDf['last_user_shop_id'] = tempDf['user_item_shop_id'].shift(1)
    tempDf['last_user_shop_id'] = tempDf['last_user_shop_id']==tempDf['user_item_shop_id']
    tempDf['last_show_time'] = tempDf['date'].shift(1)
    tempDf.loc[~tempDf.last_user_shop_id, 'last_show_time'] = np.nan
    historyShowList = []
    deltaTime = 99999999
    for same, dt, lastShowTime in tempDf[['last_user_shop_id', 'date', 'last_show_time']].values:
        if same:
            deltaTime = (dt - lastShowTime) / np.timedelta64(1, 's')
            historyShowList.append(deltaTime)
            deltaTime = 99999999
        else:
            historyShowList.append(deltaTime)
    tempDf['userShop_lastClickDeltaTime'] = historyShowList
    train_df_1_copy = train_df_1_copy.merge(tempDf[['user_item_shop_id', 'date', 'userShop_lastClickDeltaTime']], how='left', on=['user_item_shop_id', 'date'])
    train_df_1['userShop_lastClickDeltaTime'] = train_df_1_copy['userShop_lastClickDeltaTime']
    train_df_2_copy = train_df_2_copy.merge(tempDf[['user_item_shop_id', 'date', 'userShop_lastClickDeltaTime']], how='left', on=['user_item_shop_id', 'date'])
    train_df_2['userShop_lastClickDeltaTime'] = train_df_2_copy['userShop_lastClickDeltaTime']
    train_df_3_copy = train_df_3_copy.merge(tempDf[['user_item_shop_id', 'date', 'userShop_lastClickDeltaTime']], how='left', on=['user_item_shop_id', 'date'])
    train_df_3['userShop_lastClickDeltaTime'] = train_df_3_copy['userShop_lastClickDeltaTime']
    test_df_copy = pd.merge(test_df_copy, tempDf[['user_item_shop_id', 'date', 'userShop_lastClickDeltaTime']], on = ['user_item_shop_id', 'date'], how='left')
    test_df['userShop_lastClickDeltaTime'] = test_df_copy['userShop_lastClickDeltaTime']
    print(len(train_df_1))
    return train_df_1, train_df_2, train_df_3, test_df

train_df_1, train_df_2, train_df_3, test_df = getUserShopLastClickDeltaTime(train_df_1, train_df_2, train_df_3, test_df)


622020


/usr/local/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/usr/local/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
/usr/local/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.py

622020


In [21]:
# 统计某用户距离上次点击相同根类目的时间
def getUserFirstCategoryLastClickDeltaTime(train_df_1, train_df_2, train_df_3, test_df):
    print(len(train_df_1))
    train_df_1_copy = train_df_1[['user_id', 'real_first_category', 'date', 'instance_id']]
    train_df_2_copy = train_df_2[['user_id', 'real_first_category', 'date', 'instance_id']]
    train_df_3_copy = train_df_3[['user_id', 'real_first_category', 'date', 'instance_id']]
    test_df_copy = test_df[['user_id', 'real_first_category', 'date', 'instance_id']]
    train_df_1_copy['user_id_str'] = train_df_1_copy['user_id'].map(lambda x: str(x))
    train_df_1_copy['real_first_category_str'] = train_df_1_copy['real_first_category'].map(lambda x: str(x))
    train_df_1_copy['user_real_first_category'] = train_df_1_copy['user_id_str'] + train_df_1_copy['real_first_category_str']
    train_df_2_copy['user_id_str'] = train_df_2_copy['user_id'].map(lambda x: str(x))
    train_df_2_copy['real_first_category_str'] = train_df_2_copy['real_first_category'].map(lambda x: str(x))
    train_df_2_copy['user_real_first_category'] = train_df_2_copy['user_id_str'] + train_df_2_copy['real_first_category_str']
    train_df_3_copy['user_id_str'] = train_df_3_copy['user_id'].map(lambda x: str(x))
    train_df_3_copy['real_first_category_str'] = train_df_3_copy['real_first_category'].map(lambda x: str(x))
    train_df_3_copy['user_real_first_category'] = train_df_3_copy['user_id_str'] + train_df_3_copy['real_first_category_str']
    test_df_copy['user_id_str'] = test_df_copy['user_id'].map(lambda x: str(x))
    test_df_copy['real_first_category_str'] = test_df_copy['real_first_category'].map(lambda x: str(x))
    test_df_copy['user_real_first_category'] = test_df_copy['user_id_str'] + test_df_copy['real_first_category_str']
    temp_df = pd.concat([train_df_1_copy, train_df_2_copy, train_df_3_copy, test_df_copy])
    tempDf = pd.pivot_table(temp_df, index=['user_real_first_category', 'date'], values=['instance_id'], aggfunc=len)
    tempDf.columns = ['show']
    tempDf.reset_index(inplace=True)
    tempDf['last_user_real_first_category'] = tempDf['user_real_first_category'].shift(1)
    tempDf['last_user_real_first_category'] = tempDf['last_user_real_first_category']==tempDf['user_real_first_category']
    tempDf['last_show_time'] = tempDf['date'].shift(1)
    tempDf.loc[~tempDf.last_user_real_first_category, 'last_show_time'] = np.nan
    historyShowList = []
    deltaTime = 99999999
    for same, dt, lastShowTime in tempDf[['last_user_real_first_category', 'date', 'last_show_time']].values:
        if same:
            deltaTime = (dt - lastShowTime) / np.timedelta64(1, 's')
            historyShowList.append(deltaTime)
            deltaTime = 99999999
        else:
            historyShowList.append(deltaTime)
    tempDf['userFirstCategory_lastClickDeltaTime'] = historyShowList
    train_df_1_copy = train_df_1_copy.merge(tempDf[['user_real_first_category', 'date', 'userFirstCategory_lastClickDeltaTime']], how='left', on=['user_real_first_category', 'date'])
    train_df_1['userFirstCategory_lastClickDeltaTime'] = train_df_1_copy['userFirstCategory_lastClickDeltaTime']
    train_df_2_copy = train_df_2_copy.merge(tempDf[['user_real_first_category', 'date', 'userFirstCategory_lastClickDeltaTime']], how='left', on=['user_real_first_category', 'date'])
    train_df_2['userFirstCategory_lastClickDeltaTime'] = train_df_2_copy['userFirstCategory_lastClickDeltaTime']
    train_df_3_copy = train_df_3_copy.merge(tempDf[['user_real_first_category', 'date', 'userFirstCategory_lastClickDeltaTime']], how='left', on=['user_real_first_category', 'date'])
    train_df_3['userFirstCategory_lastClickDeltaTime'] = train_df_3_copy['userFirstCategory_lastClickDeltaTime']
    test_df_copy = pd.merge(test_df_copy, tempDf[['user_real_first_category', 'date', 'userFirstCategory_lastClickDeltaTime']], on = ['user_real_first_category', 'date'], how='left')
    test_df['userFirstCategory_lastClickDeltaTime'] = test_df_copy['userFirstCategory_lastClickDeltaTime']
    print(len(train_df_1))
    return train_df_1, train_df_2, train_df_3, test_df

train_df_1, train_df_2, train_df_3, test_df = getUserFirstCategoryLastClickDeltaTime(train_df_1, train_df_2, train_df_3, test_df)


622020


/usr/local/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/usr/local/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
/usr/local/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.py

622020


In [22]:
# 统计某用户距离上次点击相同叶子类目的时间
def getUserLastCategoryLastClickDeltaTime(train_df_1, train_df_2, train_df_3, test_df):
    print(len(train_df_1))
    train_df_1_copy = train_df_1[['user_id', 'real_last_category', 'date', 'instance_id']]
    train_df_2_copy = train_df_2[['user_id', 'real_last_category', 'date', 'instance_id']]
    train_df_3_copy = train_df_3[['user_id', 'real_last_category', 'date', 'instance_id']]
    test_df_copy = test_df[['user_id', 'real_last_category', 'date', 'instance_id']]
    train_df_1_copy['user_id_str'] = train_df_1_copy['user_id'].map(lambda x: str(x))
    train_df_1_copy['real_last_category_str'] = train_df_1_copy['real_last_category'].map(lambda x: str(x))
    train_df_1_copy['user_real_last_category'] = train_df_1_copy['user_id_str'] + train_df_1_copy['real_last_category_str']
    train_df_2_copy['user_id_str'] = train_df_2_copy['user_id'].map(lambda x: str(x))
    train_df_2_copy['real_last_category_str'] = train_df_2_copy['real_last_category'].map(lambda x: str(x))
    train_df_2_copy['user_real_last_category'] = train_df_2_copy['user_id_str'] + train_df_2_copy['real_last_category_str']
    train_df_3_copy['user_id_str'] = train_df_3_copy['user_id'].map(lambda x: str(x))
    train_df_3_copy['real_last_category_str'] = train_df_3_copy['real_last_category'].map(lambda x: str(x))
    train_df_3_copy['user_real_last_category'] = train_df_3_copy['user_id_str'] + train_df_3_copy['real_last_category_str']
    test_df_copy['user_id_str'] = test_df_copy['user_id'].map(lambda x: str(x))
    test_df_copy['real_last_category_str'] = test_df_copy['real_last_category'].map(lambda x: str(x))
    test_df_copy['user_real_last_category'] = test_df_copy['user_id_str'] + test_df_copy['real_last_category_str']
    temp_df = pd.concat([train_df_1_copy, train_df_2_copy, train_df_3_copy, test_df_copy])
    tempDf = pd.pivot_table(temp_df, index=['user_real_last_category', 'date'], values=['instance_id'], aggfunc=len)
    tempDf.columns = ['show']
    tempDf.reset_index(inplace=True)
    tempDf['last_user_real_last_category'] = tempDf['user_real_last_category'].shift(1)
    tempDf['last_user_real_last_category'] = tempDf['last_user_real_last_category']==tempDf['user_real_last_category']
    tempDf['last_show_time'] = tempDf['date'].shift(1)
    tempDf.loc[~tempDf.last_user_real_last_category, 'last_show_time'] = np.nan
    historyShowList = []
    deltaTime = 99999999
    for same, dt, lastShowTime in tempDf[['last_user_real_last_category', 'date', 'last_show_time']].values:
        if same:
            deltaTime = (dt - lastShowTime) / np.timedelta64(1, 's')
            historyShowList.append(deltaTime)
            deltaTime = 99999999
        else:
            historyShowList.append(deltaTime)
    tempDf['userLastCategory_lastClickDeltaTime'] = historyShowList
    train_df_1_copy = train_df_1_copy.merge(tempDf[['user_real_last_category', 'date', 'userLastCategory_lastClickDeltaTime']], how='left', on=['user_real_last_category', 'date'])
    train_df_1['userLastCategory_lastClickDeltaTime'] = train_df_1_copy['userLastCategory_lastClickDeltaTime']
    train_df_2_copy = train_df_2_copy.merge(tempDf[['user_real_last_category', 'date', 'userLastCategory_lastClickDeltaTime']], how='left', on=['user_real_last_category', 'date'])
    train_df_2['userLastCategory_lastClickDeltaTime'] = train_df_2_copy['userLastCategory_lastClickDeltaTime']
    train_df_3_copy = train_df_3_copy.merge(tempDf[['user_real_last_category', 'date', 'userLastCategory_lastClickDeltaTime']], how='left', on=['user_real_last_category', 'date'])
    train_df_3['userLastCategory_lastClickDeltaTime'] = train_df_3_copy['userLastCategory_lastClickDeltaTime']
    test_df_copy = pd.merge(test_df_copy, tempDf[['user_real_last_category', 'date', 'userLastCategory_lastClickDeltaTime']], on = ['user_real_last_category', 'date'], how='left')
    test_df['userLastCategory_lastClickDeltaTime'] = test_df_copy['userLastCategory_lastClickDeltaTime']
    print(len(train_df_1))
    return train_df_1, train_df_2, train_df_3, test_df

train_df_1, train_df_2, train_df_3, test_df = getUserLastCategoryLastClickDeltaTime(train_df_1, train_df_2, train_df_3, test_df)


622020


/usr/local/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/usr/local/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
/usr/local/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.py

622020


In [23]:
#定义添加每个小时转化率特征，test数据集采用预测方法填充
def getHourTradeRate(train_df_1, train_df_2, train_df_3, test_df):
    train_df_all = pd.concat([train_df_1, train_df_2, train_df_3])
    train_df_hour_pivot_table_all = pd.pivot_table(train_df_all[['day', 'hour', 'instance_id']], index=['day', 'hour'], values=['instance_id'], aggfunc=len)
    train_df_hour_pivot_table_all.reset_index(inplace=True)
    train_df_hour_pivot_table_all.rename(columns={'instance_id' : 'all_click_number'}, inplace=True)

    train_df_hour_pivot_table_buy = pd.pivot_table(train_df_all[['day', 'hour', 'instance_id']][train_df_all.is_trade == 1], index=['day', 'hour'], values=['instance_id'], aggfunc=len)
    train_df_hour_pivot_table_buy.reset_index(inplace=True)
    train_df_hour_pivot_table_buy.rename(columns={'instance_id' : 'all_buy_number'}, inplace=True)

    train_df_hour_pivot_table_all = pd.merge(train_df_hour_pivot_table_all, train_df_hour_pivot_table_buy, on=['day', 'hour'], how='left')
    train_df_hour_pivot_table_all['rate'] = train_df_hour_pivot_table_all['all_buy_number'] / train_df_hour_pivot_table_all['all_click_number']

    print(train_df_hour_pivot_table_all[train_df_hour_pivot_table_all.day == 7])

    hour_11_normal_mean = train_df_hour_pivot_table_all['rate'][train_df_hour_pivot_table_all.hour == 11].mean()
    hour_11_diff = train_df_hour_pivot_table_all['rate'][(train_df_hour_pivot_table_all.hour == 11) & (train_df_hour_pivot_table_all.day == 7)] - hour_11_normal_mean
    print(hour_11_diff)

    hour_nextHalf_normal_mean = pd.pivot_table(train_df_hour_pivot_table_all[['hour', 'rate']][(train_df_hour_pivot_table_all.hour > 11) & ((train_df_hour_pivot_table_all.day == 31) | (train_df_hour_pivot_table_all.day <= 4))], index=['hour'], values=['rate'], aggfunc=mean)
    hour_nextHalf_normal_mean.reset_index(inplace=True)
    hour_nextHalf_normal_mean['special_rate'] = hour_nextHalf_normal_mean['rate'] + hour_11_diff.values
    print(hour_nextHalf_normal_mean.head)
    print(hour_nextHalf_normal_mean['special_rate'].mean())

    train_df_hour_pivot_table_all.rename(columns={'rate' : 'hour_rate'}, inplace=True)
    hour_nextHalf_normal_mean.rename(columns={'special_rate' : 'hour_rate'}, inplace=True)

    train_df_1 = pd.merge(train_df_1, train_df_hour_pivot_table_all[['day', 'hour', 'hour_rate']], on=['day', 'hour'], how='left')
    train_df_2 = pd.merge(train_df_2, train_df_hour_pivot_table_all[['day', 'hour', 'hour_rate']], on=['day', 'hour'], how='left')
    train_df_3 = pd.merge(train_df_3, train_df_hour_pivot_table_all[['day', 'hour', 'hour_rate']], on=['day', 'hour'], how='left')
    test_df = pd.merge(test_df, hour_nextHalf_normal_mean[['hour', 'hour_rate']], on=['hour'], how='left')
    print(train_df_3[['day', 'hour', 'hour_rate']].head(10))
    print(test_df[['day', 'hour', 'hour_rate']].head(10))
    return train_df_1, train_df_2, train_df_3, test_df

train_df_1, train_df_2, train_df_3, test_df = getHourTradeRate(train_df_1, train_df_2, train_df_3, test_df)


     day  hour  all_click_number  all_buy_number      rate
144    7     0             17744             894  0.050383
145    7     1              9644             421  0.043654
146    7     2              4373             202  0.046193
147    7     3              2550              98  0.038431
148    7     4              1911              94  0.049189
149    7     5              2396             108  0.045075
150    7     6              5552             227  0.040886
151    7     7             10666             466  0.043690
152    7     8             12504             575  0.045985
153    7     9             13383             667  0.049839
154    7    10             14359             658  0.045825
155    7    11             12636             533  0.042181
155    0.024274
Name: rate, dtype: float64
<bound method NDFrame.head of     hour      rate  special_rate
0     12  0.016176      0.040450
1     13  0.015343      0.039617
2     14  0.012466      0.036740
3     15  0.013358      0.03

In [24]:
# 定义获取某种店铺，品牌，城市对应商品种类，用户数的函数
def getCorrespondNumber(train_df_1, train_df_2, train_df_3, test_df, colName1, colName2, newColName):

    df = pd.concat([train_df_1[[colName1, colName2, 'instance_id']], train_df_2[[colName1, colName2, 'instance_id']], train_df_3[[colName1, colName2, 'instance_id']], test_df[[colName1, colName2, 'instance_id']]])
    temp_df = df[[colName1, colName2, 'instance_id']]
    tempDf = temp_df.sort_values(by=colName1, ascending=False)
    tempDf['last_' + colName1] = tempDf[colName1].shift(1)
    tempDf['same'] = tempDf['last_' + colName1]==tempDf[colName1]
#     print(tempDf.head(10))
    colName1List = []
    countList = []
    colName2Set = set()
    for same, col2, last_col1 in tempDf[['same', colName2, 'last_' + colName1]].values:
        if same:
            colName2Set.add(col2)
        else:
            colName1List.append(last_col1)
            countList.append(len(colName2Set))
            colName2Set = {col2}
    #处理最后一行数据
    last_col1 = tempDf.iloc[-1][colName1]
    last_count = len(colName2Set)
    colName1List.append(last_col1)
    countList.append(last_count)

    #将结果组合到tempDf中
    result_df = {colName1: colName1List, newColName: countList}
    result_df = DataFrame(result_df)
    result_df = result_df[1:]

    tempDf = tempDf.drop_duplicates([colName1])
    tempDf[newColName] = result_df[newColName].values

    print(len(train_df_1))
    train_df_1 = pd.merge(train_df_1, tempDf[[colName1, newColName]], on=[colName1], how='left')
    print(len(train_df_1))
    train_df_2 = pd.merge(train_df_2, tempDf[[colName1, newColName]], on=[colName1], how='left')
    train_df_3 = pd.merge(train_df_3, tempDf[[colName1, newColName]], on=[colName1], how='left')
    test_df = pd.merge(test_df, tempDf[[colName1, newColName]], on=[colName1], how='left')

    return train_df_1, train_df_2, train_df_3, test_df

train_df_1, train_df_2, train_df_3, test_df = getCorrespondNumber(train_df_1, train_df_2, train_df_3, test_df, 'shop_id', 'item_id', 'shop_item_classNumber')
train_df_1, train_df_2, train_df_3, test_df = getCorrespondNumber(train_df_1, train_df_2, train_df_3, test_df, 'item_brand_id', 'item_id', 'brand_item_classNumber')
train_df_1, train_df_2, train_df_3, test_df = getCorrespondNumber(train_df_1, train_df_2, train_df_3, test_df, 'item_city_id', 'item_id', 'city_item_classNumber')
train_df_1, train_df_2, train_df_3, test_df = getCorrespondNumber(train_df_1, train_df_2, train_df_3, test_df, 'shop_id', 'user_id', 'shop_user_classNumber')
train_df_1, train_df_2, train_df_3, test_df = getCorrespondNumber(train_df_1, train_df_2, train_df_3, test_df, 'item_brand_id', 'user_id', 'brand_user_classNumber')
train_df_1, train_df_2, train_df_3, test_df = getCorrespondNumber(train_df_1, train_df_2, train_df_3, test_df, 'item_city_id', 'user_id', 'city_user_classNumber')



622020
622020
622020
622020
622020
622020
622020
622020
622020
622020
622020
622020


In [25]:
# 定义获取用户浏览过商品的平均值，众数，中位数，最大值，最小值
def getUserItemStatFuture(train_df_1, train_df_2, train_df_3, test_df, colName):
    df = pd.concat([train_df_1, train_df_2, train_df_3, test_df])
    df_user_item_pivot_table = pd.pivot_table(df, index=['user_id'], values=[colName], aggfunc=[np.mean, np.max, np.min, np.median])
    df_user_item_pivot_table.reset_index(inplace=True)
    df_user_item_pivot_table.columns = ['user_id', colName + '_mean', colName + '_max', colName + '_min', colName + '_median']

    df_mode_pivot_table = pd.pivot_table(df[['user_id', colName, 'instance_id']], index=['user_id', colName], values=['instance_id'], aggfunc=len)
    df_mode_pivot_table.reset_index(inplace=True)
    df_mode_pivot_table = df_mode_pivot_table.sort_values(by=['user_id', 'instance_id'], ascending=False)
    df_mode_pivot_table = df_mode_pivot_table.drop_duplicates(['user_id'])
    df_mode_pivot_table.rename(columns={colName:colName + '_mode'}, inplace=True)

    train_df_1 = pd.merge(train_df_1, df_user_item_pivot_table, on=['user_id'], how='left')
    train_df_1 = pd.merge(train_df_1, df_mode_pivot_table[['user_id', colName + '_mode']], on=['user_id'], how='left')
    train_df_2 = pd.merge(train_df_2, df_user_item_pivot_table, on=['user_id'], how='left')
    train_df_2 = pd.merge(train_df_2, df_mode_pivot_table[['user_id', colName + '_mode']], on=['user_id'], how='left')
    train_df_3 = pd.merge(train_df_3, df_user_item_pivot_table, on=['user_id'], how='left')
    train_df_3 = pd.merge(train_df_3, df_mode_pivot_table[['user_id', colName + '_mode']], on=['user_id'], how='left')
    test_df = pd.merge(test_df, df_user_item_pivot_table, on=['user_id'], how='left')
    test_df = pd.merge(test_df, df_mode_pivot_table[['user_id', colName + '_mode']], on=['user_id'], how='left')
    return train_df_1, train_df_2, train_df_3, test_df

train_df_1, train_df_2, train_df_3, test_df = getUserItemStatFuture(train_df_1, train_df_2, train_df_3, test_df, 'item_price_level')
train_df_1, train_df_2, train_df_3, test_df = getUserItemStatFuture(train_df_1, train_df_2, train_df_3, test_df, 'item_sales_level')


In [26]:
# 统计历史记录中某用户后面是否有点击相同商品
def getIsClickSameItemLater(train_df_1, train_df_2, train_df_3, test_df):
    print(len(train_df_1))
    train_df_1_copy = train_df_1[['user_id', 'item_id', 'date', 'instance_id']]
    train_df_2_copy = train_df_2[['user_id', 'item_id', 'date', 'instance_id']]
    train_df_3_copy = train_df_3[['user_id', 'item_id', 'date', 'instance_id']]
    test_df_copy = test_df[['user_id', 'item_id', 'date', 'instance_id']]
    train_df_1_copy['user_id_str'] = train_df_1_copy['user_id'].map(lambda x: str(x))
    train_df_1_copy['item_id_str'] = train_df_1_copy['item_id'].map(lambda x: str(x))
    train_df_1_copy['user_item_id'] = train_df_1_copy['user_id_str'] + train_df_1_copy['item_id_str']
    train_df_2_copy['user_id_str'] = train_df_2_copy['user_id'].map(lambda x: str(x))
    train_df_2_copy['item_id_str'] = train_df_2_copy['item_id'].map(lambda x: str(x))
    train_df_2_copy['user_item_id'] = train_df_2_copy['user_id_str'] + train_df_2_copy['item_id_str']
    train_df_3_copy['user_id_str'] = train_df_3_copy['user_id'].map(lambda x: str(x))
    train_df_3_copy['item_id_str'] = train_df_3_copy['item_id'].map(lambda x: str(x))
    train_df_3_copy['user_item_id'] = train_df_3_copy['user_id_str'] + train_df_3_copy['item_id_str']
    test_df_copy['user_id_str'] = test_df_copy['user_id'].map(lambda x: str(x))
    test_df_copy['item_id_str'] = test_df_copy['item_id'].map(lambda x: str(x))
    test_df_copy['user_item_id'] = test_df_copy['user_id_str'] + test_df_copy['item_id_str']
    temp_df = pd.concat([train_df_1_copy, train_df_2_copy, train_df_3_copy, test_df_copy])
    tempDf = pd.pivot_table(temp_df, index=['user_item_id', 'date'], values=['instance_id'], aggfunc=len)
    tempDf.columns = ['show']
    tempDf.reset_index(inplace=True)
    tempDf = tempDf.sort_values(by=['user_item_id', 'date'], ascending=False)
    tempDf['last_user_item_id'] = tempDf['user_item_id'].shift(1)
    tempDf['last_user_item_id'] = tempDf['last_user_item_id']==tempDf['user_item_id']
    tempDf['last_show_time'] = tempDf['date'].shift(1)
    tempDf.loc[~tempDf.last_user_item_id, 'last_show_time'] = np.nan
    historyShowList = []
    for same, dt in tempDf[['last_user_item_id','date']].values:
        if same:
            historyShowList.append(1)
        else:
            historyShowList.append(0)
    tempDf['is_later_clickSameItem'] = historyShowList
    print(tempDf[['last_user_item_id', 'date', 'is_later_clickSameItem']].head(20))
    train_df_1_copy = train_df_1_copy.merge(tempDf[['user_item_id', 'date', 'is_later_clickSameItem']], how='left', on=['user_item_id', 'date'])
    train_df_1['is_later_clickSameItem'] = train_df_1_copy['is_later_clickSameItem']
    train_df_2_copy = train_df_2_copy.merge(tempDf[['user_item_id', 'date', 'is_later_clickSameItem']], how='left', on=['user_item_id', 'date'])
    train_df_2['is_later_clickSameItem'] = train_df_2_copy['is_later_clickSameItem']
    train_df_3_copy = train_df_3_copy.merge(tempDf[['user_item_id', 'date', 'is_later_clickSameItem']], how='left', on=['user_item_id', 'date'])
    train_df_3['is_later_clickSameItem'] = train_df_3_copy['is_later_clickSameItem']
    test_df_copy = pd.merge(test_df_copy, tempDf[['user_item_id', 'date', 'is_later_clickSameItem']], on = ['user_item_id', 'date'], how='left')
    test_df['is_later_clickSameItem'] = test_df_copy['is_later_clickSameItem']
    print(len(train_df_1))
    return train_df_1, train_df_2, train_df_3, test_df

train_df_1, train_df_2, train_df_3, test_df = getIsClickSameItemLater(train_df_1, train_df_2, train_df_3, test_df)


622020


/usr/local/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/usr/local/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
/usr/local/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.py

         last_user_item_id                date  is_later_clickSameItem
1157354              False 2018-09-05 07:17:07                       0
1157353              False 2018-09-04 08:32:28                       0
1157352              False 2018-09-06 08:58:40                       0
1157351              False 2018-08-31 22:50:39                       0
1157350              False 2018-09-01 11:02:01                       0
1157349              False 2018-09-05 22:15:12                       0
1157348              False 2018-09-07 08:57:01                       0
1157347              False 2018-09-03 09:51:51                       0
1157346              False 2018-09-07 12:03:01                       0
1157345              False 2018-09-05 15:44:28                       0
1157344              False 2018-09-01 08:51:15                       0
1157343              False 2018-09-01 20:47:46                       0
1157342              False 2018-09-03 15:17:26                       0
115734

In [27]:
# 统计历史记录中某用户后面是否有点击相同叶子类目商品
def getIsClickSameLastCategoryLater(train_df_1, train_df_2, train_df_3, test_df):
    print(len(train_df_1))
    train_df_1_copy = train_df_1[['user_id', 'real_last_category', 'date', 'instance_id']]
    train_df_2_copy = train_df_2[['user_id', 'real_last_category', 'date', 'instance_id']]
    train_df_3_copy = train_df_3[['user_id', 'real_last_category', 'date', 'instance_id']]
    test_df_copy = test_df[['user_id', 'real_last_category', 'date', 'instance_id']]
    train_df_1_copy['user_id_str'] = train_df_1_copy['user_id'].map(lambda x: str(x))
    train_df_1_copy['real_last_category_str'] = train_df_1_copy['real_last_category'].map(lambda x: str(x))
    train_df_1_copy['user_real_last_category'] = train_df_1_copy['user_id_str'] + train_df_1_copy['real_last_category_str']
    train_df_2_copy['user_id_str'] = train_df_2_copy['user_id'].map(lambda x: str(x))
    train_df_2_copy['real_last_category_str'] = train_df_2_copy['real_last_category'].map(lambda x: str(x))
    train_df_2_copy['user_real_last_category'] = train_df_2_copy['user_id_str'] + train_df_2_copy['real_last_category_str']
    train_df_3_copy['user_id_str'] = train_df_3_copy['user_id'].map(lambda x: str(x))
    train_df_3_copy['real_last_category_str'] = train_df_3_copy['real_last_category'].map(lambda x: str(x))
    train_df_3_copy['user_real_last_category'] = train_df_3_copy['user_id_str'] + train_df_3_copy['real_last_category_str']
    test_df_copy['user_id_str'] = test_df_copy['user_id'].map(lambda x: str(x))
    test_df_copy['real_last_category_str'] = test_df_copy['real_last_category'].map(lambda x: str(x))
    test_df_copy['user_real_last_category'] = test_df_copy['user_id_str'] + test_df_copy['real_last_category_str']
    temp_df = pd.concat([train_df_1_copy, train_df_2_copy, train_df_3_copy, test_df_copy])
    tempDf = pd.pivot_table(temp_df, index=['user_real_last_category', 'date'], values=['instance_id'], aggfunc=len)
    tempDf.columns = ['show']
    tempDf.reset_index(inplace=True)
    tempDf = tempDf.sort_values(by=['user_real_last_category', 'date'], ascending=False)
    tempDf['last_user_real_last_category'] = tempDf['user_real_last_category'].shift(1)
    tempDf['last_user_real_last_category'] = tempDf['last_user_real_last_category']==tempDf['user_real_last_category']
    tempDf['last_show_time'] = tempDf['date'].shift(1)
    tempDf.loc[~tempDf.last_user_real_last_category, 'last_show_time'] = np.nan
    historyShowList = []
    for same, dt in tempDf[['last_user_real_last_category','date']].values:
        if same:
            historyShowList.append(1)
        else:
            historyShowList.append(0)
    tempDf['is_later_clickSameLastCategory'] = historyShowList
    train_df_1_copy = train_df_1_copy.merge(tempDf[['user_real_last_category', 'date', 'is_later_clickSameLastCategory']], how='left', on=['user_real_last_category', 'date'])
    train_df_1['is_later_clickSameLastCategory'] = train_df_1_copy['is_later_clickSameLastCategory']
    train_df_2_copy = train_df_2_copy.merge(tempDf[['user_real_last_category', 'date', 'is_later_clickSameLastCategory']], how='left', on=['user_real_last_category', 'date'])
    train_df_2['is_later_clickSameLastCategory'] = train_df_2_copy['is_later_clickSameLastCategory']
    train_df_3_copy = train_df_3_copy.merge(tempDf[['user_real_last_category', 'date', 'is_later_clickSameLastCategory']], how='left', on=['user_real_last_category', 'date'])
    train_df_3['is_later_clickSameLastCategory'] = train_df_3_copy['is_later_clickSameLastCategory']
    test_df_copy = pd.merge(test_df_copy, tempDf[['user_real_last_category', 'date', 'is_later_clickSameLastCategory']], on = ['user_real_last_category', 'date'], how='left')
    test_df['is_later_clickSameLastCategory'] = test_df_copy['is_later_clickSameLastCategory']
    print(len(train_df_1))
    return train_df_1, train_df_2, train_df_3, test_df

train_df_1, train_df_2, train_df_3, test_df = getIsClickSameLastCategoryLater(train_df_1, train_df_2, train_df_3, test_df)


622020


/usr/local/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/usr/local/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
/usr/local/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.py

622020


In [28]:
# 统计历史记录中某用户后面点击相同商品的个数
def getClickSameItemLaterNumber(train_df_1, train_df_2, train_df_3, test_df):
    print(len(train_df_1))
    train_df_1_copy = train_df_1[['user_id', 'item_id', 'date', 'instance_id']]
    train_df_2_copy = train_df_2[['user_id', 'item_id', 'date', 'instance_id']]
    train_df_3_copy = train_df_3[['user_id', 'item_id', 'date', 'instance_id']]
    test_df_copy = test_df[['user_id', 'item_id', 'date', 'instance_id']]
    train_df_1_copy['user_id_str'] = train_df_1_copy['user_id'].map(lambda x: str(x))
    train_df_1_copy['item_id_str'] = train_df_1_copy['item_id'].map(lambda x: str(x))
    train_df_1_copy['user_item_id'] = train_df_1_copy['user_id_str'] + train_df_1_copy['item_id_str']
    train_df_2_copy['user_id_str'] = train_df_2_copy['user_id'].map(lambda x: str(x))
    train_df_2_copy['item_id_str'] = train_df_2_copy['item_id'].map(lambda x: str(x))
    train_df_2_copy['user_item_id'] = train_df_2_copy['user_id_str'] + train_df_2_copy['item_id_str']
    train_df_3_copy['user_id_str'] = train_df_3_copy['user_id'].map(lambda x: str(x))
    train_df_3_copy['item_id_str'] = train_df_3_copy['item_id'].map(lambda x: str(x))
    train_df_3_copy['user_item_id'] = train_df_3_copy['user_id_str'] + train_df_3_copy['item_id_str']
    test_df_copy['user_id_str'] = test_df_copy['user_id'].map(lambda x: str(x))
    test_df_copy['item_id_str'] = test_df_copy['item_id'].map(lambda x: str(x))
    test_df_copy['user_item_id'] = test_df_copy['user_id_str'] + test_df_copy['item_id_str']
    temp_df = pd.concat([train_df_1_copy, train_df_2_copy, train_df_3_copy, test_df_copy])
    tempDf = pd.pivot_table(temp_df, index=['user_item_id', 'date'], values=['instance_id'], aggfunc=len)
    tempDf.columns = ['show']
    tempDf.reset_index(inplace=True)
    tempDf = tempDf.sort_values(by=['user_item_id', 'date'], ascending=False)
    tempDf['last_user_item_id'] = tempDf['user_item_id'].shift(1)
    tempDf['last_user_item_id'] = tempDf['last_user_item_id']==tempDf['user_item_id']
    tempDf['last_show_time'] = tempDf['date'].shift(1)
    tempDf.loc[~tempDf.last_user_item_id, 'last_show_time'] = np.nan
    historyShowList = []
    historyShowTemp = {}
    for same, dt in tempDf[['last_user_item_id','date']].values:
        if same:
            if len(historyShowTemp) > 0:
                historyShowList.append(len(historyShowTemp))
                historyShowTemp[dt] = same
            else:
                historyShowList.append(0)
                historyShowTemp[dt] = same
        else:
            historyShowList.append(0)
            historyShowTemp = {dt:same}
    tempDf['later_clickSameItem_count'] = historyShowList
    train_df_1_copy = train_df_1_copy.merge(tempDf[['user_item_id', 'date', 'later_clickSameItem_count']], how='left', on=['user_item_id', 'date'])
    train_df_1['later_clickSameItem_count'] = train_df_1_copy['later_clickSameItem_count']
    train_df_2_copy = train_df_2_copy.merge(tempDf[['user_item_id', 'date', 'later_clickSameItem_count']], how='left', on=['user_item_id', 'date'])
    train_df_2['later_clickSameItem_count'] = train_df_2_copy['later_clickSameItem_count']
    train_df_3_copy = train_df_3_copy.merge(tempDf[['user_item_id', 'date', 'later_clickSameItem_count']], how='left', on=['user_item_id', 'date'])
    train_df_3['later_clickSameItem_count'] = train_df_3_copy['later_clickSameItem_count']
    test_df_copy = pd.merge(test_df_copy, tempDf[['user_item_id', 'date', 'later_clickSameItem_count']], on = ['user_item_id', 'date'], how='left')
    test_df['later_clickSameItem_count'] = test_df_copy['later_clickSameItem_count']
    print(len(train_df_1))
    return train_df_1, train_df_2, train_df_3, test_df

train_df_1, train_df_2, train_df_3, test_df = getClickSameItemLaterNumber(train_df_1, train_df_2, train_df_3, test_df)



622020


/usr/local/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/usr/local/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
/usr/local/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.py

622020


In [29]:
# 统计历史记录中某用户后面点击相同叶子类目商品的个数
def getClickSameLastCategoryLaterNumber(train_df_1, train_df_2, train_df_3, test_df):
    print(len(train_df_1))
    train_df_1_copy = train_df_1[['user_id', 'real_last_category', 'date', 'instance_id']]
    train_df_2_copy = train_df_2[['user_id', 'real_last_category', 'date', 'instance_id']]
    train_df_3_copy = train_df_3[['user_id', 'real_last_category', 'date', 'instance_id']]
    test_df_copy = test_df[['user_id', 'real_last_category', 'date', 'instance_id']]
    train_df_1_copy['user_id_str'] = train_df_1_copy['user_id'].map(lambda x: str(x))
    train_df_1_copy['real_last_category_str'] = train_df_1_copy['real_last_category'].map(lambda x: str(x))
    train_df_1_copy['user_real_last_category'] = train_df_1_copy['user_id_str'] + train_df_1_copy['real_last_category_str']
    train_df_2_copy['user_id_str'] = train_df_2_copy['user_id'].map(lambda x: str(x))
    train_df_2_copy['real_last_category_str'] = train_df_2_copy['real_last_category'].map(lambda x: str(x))
    train_df_2_copy['user_real_last_category'] = train_df_2_copy['user_id_str'] + train_df_2_copy['real_last_category_str']
    train_df_3_copy['user_id_str'] = train_df_3_copy['user_id'].map(lambda x: str(x))
    train_df_3_copy['real_last_category_str'] = train_df_3_copy['real_last_category'].map(lambda x: str(x))
    train_df_3_copy['user_real_last_category'] = train_df_3_copy['user_id_str'] + train_df_3_copy['real_last_category_str']
    test_df_copy['user_id_str'] = test_df_copy['user_id'].map(lambda x: str(x))
    test_df_copy['real_last_category_str'] = test_df_copy['real_last_category'].map(lambda x: str(x))
    test_df_copy['user_real_last_category'] = test_df_copy['user_id_str'] + test_df_copy['real_last_category_str']
    temp_df = pd.concat([train_df_1_copy, train_df_2_copy, train_df_3_copy, test_df_copy])

    tempDf = pd.pivot_table(temp_df, index=['user_real_last_category', 'date'], values=['instance_id'], aggfunc=len)
    tempDf.columns = ['show']
    tempDf.reset_index(inplace=True)
    tempDf = tempDf.sort_values(by=['user_real_last_category', 'date'], ascending=False)
    tempDf['last_user_real_last_category'] = tempDf['user_real_last_category'].shift(1)
    tempDf['last_user_real_last_category'] = tempDf['last_user_real_last_category']==tempDf['user_real_last_category']
    tempDf['last_show_time'] = tempDf['date'].shift(1)
    tempDf.loc[~tempDf.last_user_real_last_category, 'last_show_time'] = np.nan
    historyShowList = []
    historyShowTemp = {}
    for same, dt in tempDf[['last_user_real_last_category','date']].values:
        if same:
            if len(historyShowTemp) > 0:
                historyShowList.append(len(historyShowTemp))
                historyShowTemp[dt] = same
            else:
                historyShowList.append(0)
                historyShowTemp[dt] = same
        else:
            historyShowList.append(0)
            historyShowTemp = {dt:same}
    tempDf['later_clickSameLastCategory_count'] = historyShowList

    train_df_1_copy = train_df_1_copy.merge(tempDf[['user_real_last_category', 'date', 'later_clickSameLastCategory_count']], how='left', on=['user_real_last_category', 'date'])
    train_df_1['later_clickSameLastCategory_count'] = train_df_1_copy['later_clickSameLastCategory_count']
    train_df_2_copy = train_df_2_copy.merge(tempDf[['user_real_last_category', 'date', 'later_clickSameLastCategory_count']], how='left', on=['user_real_last_category', 'date'])
    train_df_2['later_clickSameLastCategory_count'] = train_df_2_copy['later_clickSameLastCategory_count']
    train_df_3_copy = train_df_3_copy.merge(tempDf[['user_real_last_category', 'date', 'later_clickSameLastCategory_count']], how='left', on=['user_real_last_category', 'date'])
    train_df_3['later_clickSameLastCategory_count'] = train_df_3_copy['later_clickSameLastCategory_count']
    test_df_copy = pd.merge(test_df_copy, tempDf[['user_real_last_category', 'date', 'later_clickSameLastCategory_count']], on = ['user_real_last_category', 'date'], how='left')
    test_df['later_clickSameLastCategory_count'] = test_df_copy['later_clickSameLastCategory_count']
    print(len(train_df_1))
    return train_df_1, train_df_2, train_df_3, test_df

train_df_1, train_df_2, train_df_3, test_df = getClickSameLastCategoryLaterNumber(train_df_1, train_df_2, train_df_3, test_df)



622020


/usr/local/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/usr/local/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
/usr/local/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.py

622020


In [30]:
# 统计历史记录中某用户后面点击相同商品的时间间隔
def getClickSameItemLaterDeltaTime(train_df_1, train_df_2, train_df_3, test_df):
    print(len(train_df_1))
    train_df_1_copy = train_df_1[['user_id', 'item_id', 'date', 'instance_id']]
    train_df_2_copy = train_df_2[['user_id', 'item_id', 'date', 'instance_id']]
    train_df_3_copy = train_df_3[['user_id', 'item_id', 'date', 'instance_id']]
    test_df_copy = test_df[['user_id', 'item_id', 'date', 'instance_id']]
    train_df_1_copy['user_id_str'] = train_df_1_copy['user_id'].map(lambda x: str(x))
    train_df_1_copy['item_id_str'] = train_df_1_copy['item_id'].map(lambda x: str(x))
    train_df_1_copy['user_item_id'] = train_df_1_copy['user_id_str'] + train_df_1_copy['item_id_str']
    train_df_2_copy['user_id_str'] = train_df_2_copy['user_id'].map(lambda x: str(x))
    train_df_2_copy['item_id_str'] = train_df_2_copy['item_id'].map(lambda x: str(x))
    train_df_2_copy['user_item_id'] = train_df_2_copy['user_id_str'] + train_df_2_copy['item_id_str']
    train_df_3_copy['user_id_str'] = train_df_3_copy['user_id'].map(lambda x: str(x))
    train_df_3_copy['item_id_str'] = train_df_3_copy['item_id'].map(lambda x: str(x))
    train_df_3_copy['user_item_id'] = train_df_3_copy['user_id_str'] + train_df_3_copy['item_id_str']
    test_df_copy['user_id_str'] = test_df_copy['user_id'].map(lambda x: str(x))
    test_df_copy['item_id_str'] = test_df_copy['item_id'].map(lambda x: str(x))
    test_df_copy['user_item_id'] = test_df_copy['user_id_str'] + test_df_copy['item_id_str']
    temp_df = pd.concat([train_df_1_copy, train_df_2_copy, train_df_3_copy, test_df_copy])
    tempDf = pd.pivot_table(temp_df, index=['user_item_id', 'date'], values=['instance_id'], aggfunc=len)
    tempDf.columns = ['show']
    tempDf.reset_index(inplace=True)
    tempDf = tempDf.sort_values(by=['user_item_id', 'date'], ascending=False)
    tempDf['last_user_item_id'] = tempDf['user_item_id'].shift(1)
    tempDf['last_user_item_id'] = tempDf['last_user_item_id']==tempDf['user_item_id']
    tempDf['last_show_time'] = tempDf['date'].shift(1)
    tempDf.loc[~tempDf.last_user_item_id, 'last_show_time'] = np.nan
    historyShowList = []
    deltaTime = 99999999
    for same, dt, lastShowTime in tempDf[['last_user_item_id', 'date', 'last_show_time']].values:
        if same:
            deltaTime = (dt - lastShowTime) / np.timedelta64(1, 's')
            historyShowList.append(deltaTime)
            deltaTime = 99999999
        else:
            historyShowList.append(deltaTime)
    tempDf['later_clickSameItem_deltaTime'] = historyShowList

    train_df_1_copy = train_df_1_copy.merge(tempDf[['user_item_id', 'date', 'later_clickSameItem_deltaTime']], how='left', on=['user_item_id', 'date'])
    train_df_1['later_clickSameItem_deltaTime'] = train_df_1_copy['later_clickSameItem_deltaTime']
    train_df_2_copy = train_df_2_copy.merge(tempDf[['user_item_id', 'date', 'later_clickSameItem_deltaTime']], how='left', on=['user_item_id', 'date'])
    train_df_2['later_clickSameItem_deltaTime'] = train_df_2_copy['later_clickSameItem_deltaTime']
    train_df_3_copy = train_df_3_copy.merge(tempDf[['user_item_id', 'date', 'later_clickSameItem_deltaTime']], how='left', on=['user_item_id', 'date'])
    train_df_3['later_clickSameItem_deltaTime'] = train_df_3_copy['later_clickSameItem_deltaTime']
    test_df_copy = pd.merge(test_df_copy, tempDf[['user_item_id', 'date', 'later_clickSameItem_deltaTime']], on = ['user_item_id', 'date'], how='left')
    test_df['later_clickSameItem_deltaTime'] = test_df_copy['later_clickSameItem_deltaTime']
    print(len(train_df_1))
    return train_df_1, train_df_2, train_df_3, test_df

train_df_1, train_df_2, train_df_3, test_df = getClickSameItemLaterDeltaTime(train_df_1, train_df_2, train_df_3, test_df)



622020


/usr/local/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/usr/local/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
/usr/local/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.py

622020


In [31]:
# 统计历史记录中某用户后面点击相同叶子类目商品的时间间隔
def getClickSameLastCategoryLaterDeltaTime(train_df_1, train_df_2, train_df_3, test_df):
    print(len(train_df_1))
    train_df_1_copy = train_df_1[['user_id', 'real_last_category', 'date', 'instance_id']]
    train_df_2_copy = train_df_2[['user_id', 'real_last_category', 'date', 'instance_id']]
    train_df_3_copy = train_df_3[['user_id', 'real_last_category', 'date', 'instance_id']]
    test_df_copy = test_df[['user_id', 'real_last_category', 'date', 'instance_id']]
    train_df_1_copy['user_id_str'] = train_df_1_copy['user_id'].map(lambda x: str(x))
    train_df_1_copy['real_last_category_str'] = train_df_1_copy['real_last_category'].map(lambda x: str(x))
    train_df_1_copy['user_real_last_category'] = train_df_1_copy['user_id_str'] + train_df_1_copy['real_last_category_str']
    train_df_2_copy['user_id_str'] = train_df_2_copy['user_id'].map(lambda x: str(x))
    train_df_2_copy['real_last_category_str'] = train_df_2_copy['real_last_category'].map(lambda x: str(x))
    train_df_2_copy['user_real_last_category'] = train_df_2_copy['user_id_str'] + train_df_2_copy['real_last_category_str']
    train_df_3_copy['user_id_str'] = train_df_3_copy['user_id'].map(lambda x: str(x))
    train_df_3_copy['real_last_category_str'] = train_df_3_copy['real_last_category'].map(lambda x: str(x))
    train_df_3_copy['user_real_last_category'] = train_df_3_copy['user_id_str'] + train_df_3_copy['real_last_category_str']
    test_df_copy['user_id_str'] = test_df_copy['user_id'].map(lambda x: str(x))
    test_df_copy['real_last_category_str'] = test_df_copy['real_last_category'].map(lambda x: str(x))
    test_df_copy['user_real_last_category'] = test_df_copy['user_id_str'] + test_df_copy['real_last_category_str']
    temp_df = pd.concat([train_df_1_copy, train_df_2_copy, train_df_3_copy, test_df_copy])

    tempDf = pd.pivot_table(temp_df, index=['user_real_last_category', 'date'], values=['instance_id'], aggfunc=len)
    tempDf.columns = ['show']
    tempDf.reset_index(inplace=True)
    tempDf = tempDf.sort_values(by=['user_real_last_category', 'date'], ascending=False)
    tempDf['last_user_real_last_category'] = tempDf['user_real_last_category'].shift(1)
    tempDf['last_user_real_last_category'] = tempDf['last_user_real_last_category']==tempDf['user_real_last_category']
    tempDf['last_show_time'] = tempDf['date'].shift(1)
    tempDf.loc[~tempDf.last_user_real_last_category, 'last_show_time'] = np.nan

    historyShowList = []
    deltaTime = 99999999
    for same, dt, lastShowTime in tempDf[['last_user_real_last_category', 'date', 'last_show_time']].values:
        if same:
            deltaTime = (dt - lastShowTime) / np.timedelta64(1, 's')
            historyShowList.append(deltaTime)
            deltaTime = 99999999
        else:
            historyShowList.append(deltaTime)
    tempDf['later_clickSameLastCategory_deltaTime'] = historyShowList

    train_df_1_copy = train_df_1_copy.merge(tempDf[['user_real_last_category', 'date', 'later_clickSameLastCategory_deltaTime']], how='left', on=['user_real_last_category', 'date'])
    train_df_1['later_clickSameLastCategory_deltaTime'] = train_df_1_copy['later_clickSameLastCategory_deltaTime']
    train_df_2_copy = train_df_2_copy.merge(tempDf[['user_real_last_category', 'date', 'later_clickSameLastCategory_deltaTime']], how='left', on=['user_real_last_category', 'date'])
    train_df_2['later_clickSameLastCategory_deltaTime'] = train_df_2_copy['later_clickSameLastCategory_deltaTime']
    train_df_3_copy = train_df_3_copy.merge(tempDf[['user_real_last_category', 'date', 'later_clickSameLastCategory_deltaTime']], how='left', on=['user_real_last_category', 'date'])
    train_df_3['later_clickSameLastCategory_deltaTime'] = train_df_3_copy['later_clickSameLastCategory_deltaTime']
    test_df_copy = pd.merge(test_df_copy, tempDf[['user_real_last_category', 'date', 'later_clickSameLastCategory_deltaTime']], on = ['user_real_last_category', 'date'], how='left')
    test_df['later_clickSameLastCategory_deltaTime'] = test_df_copy['later_clickSameLastCategory_deltaTime']
    print(len(train_df_1))
    return train_df_1, train_df_2, train_df_3, test_df

train_df_1, train_df_2, train_df_3, test_df = getClickSameLastCategoryLaterDeltaTime(train_df_1, train_df_2, train_df_3, test_df)



622020


/usr/local/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/usr/local/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
/usr/local/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.py

622020


In [32]:
def getBaseConversionRate(future_df, test_df, colName):
    t = future_df[[colName]]
    t[colName + '_total_number'] = 1
    t = t.groupby(colName).agg('sum').reset_index()

    t_buy = future_df[[colName]][future_df.is_trade == 1]
    t_buy[colName + '_buy_number'] = 1
    t_buy = t_buy.groupby(colName).agg('sum').reset_index()

    t = pd.merge(t, t_buy, on=colName, how='left')
    t[colName + '_buy_number'] = t[colName + '_buy_number'].map(lambda x: 0 if math.isnan(x) else x)
    t['buy_origion_rate'] = t[colName + '_buy_number'] / t[colName + '_total_number']
    alpha, beta = getBayesSmoothParam(t['buy_origion_rate'])
    t[colName + '_converse_smooth_rate'] = (t[colName + '_buy_number'] + alpha) / (t[colName + '_total_number'] + alpha + beta)
#     train_df = pd.merge(train_df, t[[colName, colName + '_converse_smooth_rate']], on=colName, how='left')
#     train_df[colName + '_converse_smooth_rate'] = train_df[colName + '_converse_smooth_rate'].map(lambda x: (alpha / (alpha + beta)) if math.isnan(x) else x)

    test_df = pd.merge(test_df, t[[colName, colName + '_converse_smooth_rate', colName + '_total_number', colName + '_buy_number']], on=colName, how='left')
    test_df[colName + '_converse_smooth_rate'] = test_df[colName + '_converse_smooth_rate'].map(lambda x: (alpha / (alpha + beta)) if math.isnan(x) else x)

    test_df[colName + '_total_number'] = test_df[colName + '_total_number'].fillna(0)
    test_df, total_number_scaler = scalerFea(test_df, colName + '_total_number')

    test_df[colName + '_buy_number'] = test_df[colName + '_buy_number'].fillna(0)
    test_df, buy_number_scaler = scalerFea(test_df, colName + '_buy_number')

    return test_df

#定义对每个窗口进行操作的函数
def dealHuaChuangDataset(future_df, dataset):
    dataset = getBaseConversionRate(future_df, dataset, 'user_id')
    dataset = getBaseConversionRate(future_df, dataset, 'item_brand_id')
    dataset = getBaseConversionRate(future_df, dataset, 'item_id')
    dataset = getBaseConversionRate(future_df, dataset, 'shop_id')
    dataset = getBaseConversionRate(future_df, dataset, 'real_last_category')
    return dataset

#尝试统计两天前滑窗结果，包括商品，店铺，品牌，叶子类目的转化率，点击次数和购买次数
#首先划分数据集
future_dataset1 = train_df_1[(train_df_1.day == 31) | (train_df_1.day == 1)]
train_df_huachuang_1 = train_df_1[train_df_1.day == 2]
future_dataset2 = train_df_1[(train_df_1.day == 1) | (train_df_1.day == 2)]
train_df_huachuang_2 = train_df_1[train_df_1.day == 3]
future_dataset3 = train_df_1[(train_df_1.day == 2) | (train_df_1.day == 3)]
train_df_huachuang_3 = train_df_1[train_df_1.day == 4]
future_dataset4 = train_df_2
train_df_huachuang_4 = train_df_3

train_df_huachuang_1 = dealHuaChuangDataset(future_dataset1, train_df_huachuang_1)
train_df_huachuang_2 = dealHuaChuangDataset(future_dataset2, train_df_huachuang_2)
train_df_huachuang_3 = dealHuaChuangDataset(future_dataset3, train_df_huachuang_3)
train_df_huachuang_4 = dealHuaChuangDataset(future_dataset4, train_df_huachuang_4)
test_df = dealHuaChuangDataset(future_dataset4, test_df)


/usr/local/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


origion_rate_mean :  0.014030989941101248
origion_rate_var :  0.012976118619169384
alpha :  0.000927752497457697
beta :  0.06519391827218728
origion_rate_mean :  0.015054532085585381
origion_rate_var :  0.00492799638531314
alpha :  0.030243186444117682
beta :  1.9786659096463108
origion_rate_mean :  0.012097971377722512
origion_rate_var :  0.005571045964682445
alpha :  0.013855905552248306
beta :  1.1314522721280935
origion_rate_mean :  0.015381346774083779
origion_rate_var :  0.005934931270559113
alpha :  0.02386878255241443
beta :  1.5279317784121973
origion_rate_mean :  0.02435880595088018
origion_rate_var :  0.0031205021759647005
alpha :  0.1611555960360722
beta :  6.454751454623322
origion_rate_mean :  0.014093855829482698
origion_rate_var :  0.013014060390419257
alpha :  0.0009542696777117409
beta :  0.06675393517816029
origion_rate_mean :  0.015258331924165595
origion_rate_var :  0.004990279278552974
alpha :  0.030683846044190884
beta :  1.9802729345980574
origion_rate_mean :  0

In [34]:
print(len(test_df.columns.values))
print(test_df.columns.values)



102
['instance_id' 'item_id' 'item_brand_id' 'item_city_id' 'item_price_level'
 'item_sales_level' 'item_collected_level' 'item_pv_level' 'user_id'
 'user_gender_id' 'user_age_level' 'user_occupation_id' 'user_star_level'
 'context_id' 'context_timestamp' 'context_page_id'
 'predict_category_property' 'shop_id' 'shop_review_num_level'
 'shop_review_positive_rate' 'shop_star_level' 'shop_score_service'
 'shop_score_delivery' 'shop_score_description' 'date' 'weekday' 'day'
 'hour' 'prop_jaccard' 'prop_predict_ratio' 'prop_item_ratio'
 'match_category_proportion' 'match_property_proportion'
 'predict_category_number' 'predict_property_number' 'isFirstCategoryIn'
 'isLastCategoryIn' 'category_number' 'property_number'
 'real_first_category' 'real_last_category'
 'all_item_brand_id_click_number' 'all_item_brand_id_buy_number'
 'history_item_brand_id_rate' 'history_item_brand_id_smooth_rate'
 'all_shop_id_click_number' 'all_shop_id_buy_number' 'history_shop_id_rate'
 'history_shop_id_smooth_

In [35]:
#将2,3,4号数据抽样15%与7号当天数据结合作为训练接，拟合线上分布
train_df_234 = pd.concat([train_df_huachuang_1, train_df_huachuang_2, train_df_huachuang_3])
train_df_234 = train_df_234.sample(frac = 0.15, replace = True)

train_df = pd.concat([train_df_huachuang_4, train_df_234])


In [36]:
class XgbModel:
    def __init__(self, feaNames=None, params={}):
        self.feaNames = feaNames
        self.params = {
            'objective': 'binary:logistic',
            'eval_metric':'logloss',
            'silent': True,
            'eta': 0.1,
            'max_depth': 4,
            'gamma': 0.5,
            'subsample': 0.95,
            'colsample_bytree': 1,
            'min_child_weight': 8,
            'max_delta_step': 5,
            'lambda': 100,
        }
        for k,v in params.items():
            self.params[k] = v
        self.clf = None

    def train(self, X, y, train_size=1, test_size=0.1, verbose=True, num_boost_round=1000, early_stopping_rounds=3):
        X = X.astype(float)
        if train_size==1:
            X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size)
            X_train, y_train = X, y
        else:
            X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=train_size)
        dtrain = xgb.DMatrix(X_train, label=y_train, feature_names=self.feaNames)
        dval = xgb.DMatrix(X_test, label=y_test, feature_names=self.feaNames)
        watchlist = [(dtrain,'train'),(dval,'val')]
        clf = xgb.train(
            self.params, dtrain,
            num_boost_round = num_boost_round,
            evals = watchlist,
            early_stopping_rounds = early_stopping_rounds,
            verbose_eval=verbose
        )
        self.clf = clf

    def trainCV(self, X, y, nFold=3, verbose=True, num_boost_round=1500, early_stopping_rounds=10):
        X = X.astype(float)
        dtrain = xgb.DMatrix(X, label=y, feature_names=self.feaNames)
        cvResult = xgb.cv(
            self.params, dtrain,
            num_boost_round = num_boost_round,
            nfold = nFold,
            early_stopping_rounds = early_stopping_rounds,
            verbose_eval=verbose
        )
        clf = xgb.train(
            self.params, dtrain,
            num_boost_round = cvResult.shape[0],
        )
        self.clf = clf

    def gridSearch(self, X, y, nFold=3, verbose=1, num_boost_round=130):
        paramsGrids = {
            # 'n_estimators': [50+5*i for i in range(0,30)],
            'gamma': [0,0.01,0.05,0.1,0.5,1,5,10,50,100],
            # 'max_depth': list(range(3,10)),
            'min_child_weight': list(range(0,10)),
            'subsample': [1-0.05*i for i in range(0,8)],
            'colsample_bytree': [1-0.05*i for i in range(0,10)],
            # 'reg_alpha': [0+2*i for i in range(0,10)],
            'reg_lambda': [0+50*i for i in range(0,10)],
            'max_delta_step': [0+1*i for i in range(0,8)],
        }
        for k,v in paramsGrids.items():
            gsearch = GridSearchCV(
                estimator = xgb.XGBClassifier(
                    max_depth = self.params['max_depth'],
                    gamma = self.params['gamma'],
                    learning_rate = self.params['eta'],
                    max_delta_step = self.params['max_delta_step'],
                    min_child_weight = self.params['min_child_weight'],
                    subsample = self.params['subsample'],
                    colsample_bytree = self.params['colsample_bytree'],
                    silent = self.params['silent'],
                    reg_lambda = self.params['lambda'],
                    n_estimators = num_boost_round
                ),
                # param_grid = paramsGrids,
                param_grid = {k:v},
                scoring = 'neg_log_loss',
                cv = nFold,
                verbose = verbose,
                n_jobs = 4
            )
            gsearch.fit(X, y)
            print(pd.DataFrame(gsearch.cv_results_))
            print(gsearch.best_params_)
        exit()

    def predict(self, X):
        X = X.astype(float)
        return self.clf.predict(xgb.DMatrix(X, feature_names=self.feaNames))

    def getFeaScore(self, show=False):
        fscore = self.clf.get_score()
        feaNames = fscore.keys()
        scoreDf = pd.DataFrame(index=feaNames, columns=['importance'])
        for k,v in fscore.items():
            scoreDf.loc[k, 'importance'] = v
        if show:
            print(scoreDf.sort_index(by=['importance'], ascending=False))
        return scoreDf

# 划分训练集和测试集
def trainTestSplit(df, splitDate=pd.to_datetime('2018-09-23'), trainPeriod=3, testPeriod=1):
    trainDf = df[(df.context_timestamp<splitDate)&(df.context_timestamp>=splitDate-timedelta(days=trainPeriod))]
    testDf = df[(df.context_timestamp>=splitDate)&(df.context_timestamp<splitDate+timedelta(days=testPeriod))]
    return (trainDf, testDf)


# 统计预测误差
def countDeltaY(predictSeries, labelSeries, show=True, title='', subplot=None):
    deltaSeries = predictSeries - labelSeries
    if subplot!=None:
        plt.subplot(subplot[0], subplot[1], subplot[2])
    deltaSeries.plot(style='b-')
    plt.title(title)
    if show:
        plt.show()
    return deltaSeries

# 获取stacking下一层数据集
def getOof(clf, trainX, trainY, testX, nFold=5, stratify=False):
    oofTrain = np.zeros(trainX.shape[0])
    oofTest = np.zeros(testX.shape[0])
    oofTestSkf = np.zeros((testX.shape[0], nFold))
    if stratify:
        kf = StratifiedKFold(n_splits=nFold, shuffle=True)
    else:
        kf = KFold(n_splits=nFold, shuffle=True)
    for i, (trainIdx, testIdx) in enumerate(kf.split(trainX, trainY)):
        kfTrainX = trainX[trainIdx]
        kfTrainY = trainY[trainIdx]
        kfTestX = trainX[testIdx]
        clf.trainCV(kfTrainX, kfTrainY, verbose=False)
        oofTrain[testIdx] = clf.predict(kfTestX)
        oofTestSkf[:,i] = clf.predict(testX)
    oofTest[:] = oofTestSkf.mean(axis=1)
    return oofTrain, oofTest



In [91]:
fea = [

         'item_brand_id', 'item_city_id', 'item_price_level', 'item_sales_level',
         'item_collected_level', 'item_pv_level', 'user_gender_id',
         'user_age_level', 'user_occupation_id', 'user_star_level',
         'context_page_id',
         'shop_review_num_level', 'shop_review_positive_rate',
         'shop_star_level', 'shop_score_service', 'shop_score_delivery',
         'shop_score_description',
         'prop_jaccard', 'prop_predict_ratio', 'prop_item_ratio',
#          'match_category_proportion', 'match_property_proportion', 
#          'predict_category_number', 'predict_property_number',
#          'isLastCategoryIn', 'isFirstCategoryIn',
         'category_number', 'property_number',
         'all_item_brand_id_click_number',
         'all_item_brand_id_buy_number',
         'history_item_brand_id_smooth_rate', 'all_shop_id_click_number',
         'all_shop_id_buy_number',
         'history_shop_id_smooth_rate', 'all_item_id_click_number',
         'all_item_id_buy_number',
         'history_item_id_smooth_rate', 'lastOneHour_sameItem_count',
         'lastOneHour_sameFirstCategory_count', 'lastOneHour_sameLastCategory_count',
         'lastOneHour_sameBrand_count', 'lastOneHour_sameShop_count',
         'isLastOneHour_firstClickItem',
    #      'is_special', 'hour',
         'hour_rate',
         'userItem_lastClickDeltaTime',
         'userBrand_lastClickDeltaTime', 'userShop_lastClickDeltaTime',
         'userFirstCategory_lastClickDeltaTime',
         'userLastCategory_lastClickDeltaTime',
         'user_id_converse_smooth_rate', 'user_id_total_number', 'user_id_buy_number',
         'item_brand_id_converse_smooth_rate', 'item_brand_id_total_number',
         'item_brand_id_buy_number', 'item_id_converse_smooth_rate',
         'item_id_total_number', 'item_id_buy_number', 'shop_id_converse_smooth_rate',
         'shop_id_total_number', 'shop_id_buy_number',
         'real_last_category_converse_smooth_rate',
         'real_last_category_total_number', 'real_last_category_buy_number',
         'is_later_clickSameItem', 'is_later_clickSameLastCategory',
         'later_clickSameItem_count', 'later_clickSameLastCategory_count',
         'later_clickSameItem_deltaTime', 'later_clickSameLastCategory_deltaTime',
         'shop_item_classNumber', 'brand_item_classNumber', 'city_item_classNumber',
         'shop_user_classNumber', 'brand_user_classNumber', 'city_user_classNumber',
#          'item_price_level_mode','item_sales_level_mode', 
#          'item_price_level_mean', 'item_sales_level_mean', 
#          'item_price_level_max', 'item_sales_level_max', 
#          'item_price_level_min', 'item_sales_level_min',  
#         'item_sales_level_median', 'item_price_level_median', 
         ]

train_df_7 = train_df[train_df.day == 7]
xgbModel = XgbModel(feaNames=fea)
modelName = "xgb_fusai_a"

In [92]:
# 正式模型
startTime = datetime.datetime.now()
xgbModel.trainCV(train_df_7[fea].values, train_df_7['is_trade'].values)
xgbModel.getFeaScore(show=True)
print('training time: ', datetime.datetime.now()-startTime)


[0]	train-logloss:0.615224+0.000104327	test-logloss:0.615224+7.30677e-05
[1]	train-logloss:0.551455+0.000178574	test-logloss:0.551455+0.000157051
[2]	train-logloss:0.498491+0.000279256	test-logloss:0.498492+0.000203649
[3]	train-logloss:0.454037+0.000323484	test-logloss:0.45404+0.000297533
[4]	train-logloss:0.41641+0.000371131	test-logloss:0.416423+0.00037828
[5]	train-logloss:0.384351+0.00042417	test-logloss:0.384378+0.000453768
[6]	train-logloss:0.356898+0.000466335	test-logloss:0.356927+0.000527633
[7]	train-logloss:0.333293+0.000514004	test-logloss:0.333342+0.000598367
[8]	train-logloss:0.312938+0.000563383	test-logloss:0.313004+0.000657637
[9]	train-logloss:0.295326+0.000621354	test-logloss:0.295414+0.000712402
[10]	train-logloss:0.280041+0.000660707	test-logloss:0.280152+0.000775454
[11]	train-logloss:0.26678+0.000687478	test-logloss:0.26693+0.000853381
[12]	train-logloss:0.255275+0.000725587	test-logloss:0.255442+0.000907854
[13]	train-logloss:0.24526+0.000755557	test-logloss:0.

[113]	train-logloss:0.171339+0.000736332	test-logloss:0.177317+0.00176373
[114]	train-logloss:0.171299+0.000747797	test-logloss:0.177308+0.00175508
[115]	train-logloss:0.171252+0.000753706	test-logloss:0.177291+0.00175076
[116]	train-logloss:0.171207+0.000729101	test-logloss:0.177286+0.00174179
[117]	train-logloss:0.171143+0.000725596	test-logloss:0.177272+0.0017411
[118]	train-logloss:0.171077+0.000728007	test-logloss:0.177273+0.00173987
[119]	train-logloss:0.171033+0.000737625	test-logloss:0.177266+0.00173403
[120]	train-logloss:0.170986+0.000753448	test-logloss:0.177253+0.00171939
[121]	train-logloss:0.170933+0.00074229	test-logloss:0.17725+0.00171458
[122]	train-logloss:0.170892+0.00074622	test-logloss:0.177234+0.0017076
[123]	train-logloss:0.170833+0.00075324	test-logloss:0.177228+0.00170559
[124]	train-logloss:0.170795+0.000763774	test-logloss:0.177218+0.00170901
[125]	train-logloss:0.170738+0.000758573	test-logloss:0.177207+0.00170741
[126]	train-logloss:0.170684+0.00076057	test

/usr/local/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:105: FutureWarning: by argument to sort_index is deprecated, pls use .sort_values(by=...)


In [65]:
# 开始预测
test_df.loc[:,'predicted_score'] = xgbModel.predict(test_df[fea].values)
print('predicting time: ', datetime.datetime.now()-startTime)
print("预测结果：\n",test_df[['instance_id','predicted_score']].head())
print('预测均值：', test_df['predicted_score'].mean())


predicting time:  0:04:20.239288
预测结果：
            instance_id  predicted_score
0  7279286161489290771         0.087065
1  8639231129442069895         0.074561
2  8589809858455174680         0.025012
3  4336529059857606193         0.009611
4   599240169885361385         0.018930
预测均值： 0.04236214980483055


In [66]:
# 生成stacking数据集
train_df_7['predicted_score'] = np.nan
test_df['predicted_score'] = np.nan
train_df_7.loc[:,'predicted_score'], test_df.loc[:,'predicted_score'] = getOof(xgbModel, train_df_7[fea].values, train_df_7['is_trade'].values, test_df[fea].values, stratify=True)
print('oof training time: ', datetime.datetime.now()-startTime)
xgbModel.getFeaScore(show=True)
cost = metrics.log_loss(train_df_7['is_trade'].values, train_df_7['predicted_score'].values)
print('train loss: ', cost)
print('7th train loss', metrics.log_loss(train_df_7['is_trade'].values, train_df_7['predicted_score'].values))
print('7th train predict aver:', train_df_7['predicted_score'].mean())
print('test predict: \n',test_df[['instance_id','predicted_score']].head())
print('test predict aver:', test_df['predicted_score'].mean())


/usr/local/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


KeyboardInterrupt: 

In [ ]:
# 导出预测结果
def exportResult(df, fileName):
    df.to_csv('~/kengkeng/alimama/result/%s.txt' % fileName, sep=' ', header=True, index=False)

exportResult(test_df[['instance_id', 'predicted_score']], 'fusai_xgb_5_10_wen')